In [49]:
import transformers
import argparse
import ast
import logging
import os
import sys
import transformers
import pandas as pd
import torch
from tqdm import tqdm

from transformers import BartForConditionalGeneration, RagRetriever, RagSequenceForGeneration, RagTokenForGeneration
from transformers import logging as transformers_logging

model_kwargs = {}
model_class = RagTokenForGeneration 

model_kwargs["n_docs"] = 5         #这个参数用于控制检索条目！！！！！！

# added following to fixed params, without if and else. ln.364 @transformers/src/transformers/models/rag/retrieval_rag.py
model_kwargs["index_name"] = "custom"
model_kwargs["passages_path"] = '/root/autodl-tmp/rag-end2end-retriever/DIR/SQUAD-KB/hengqin/my_knowledge_dataset_1'    #横琴
# model_kwargs["passages_path"] = '/root/autodl-tmp/rag-end2end-retriever/DIR/baiduzhidao/squad2w'
model_kwargs["index_path"] = '/root/autodl-tmp/rag-end2end-retriever/DIR/SQUAD-KB/hengqin/my_knowledge_dataset_hnsw_index_1.faiss'    #横琴
# model_kwargs["index_path"] = '/root/autodl-tmp/rag-end2end-retriever/DIR/baiduzhidao/squad2w.faiss'     #百度9k是search2w，百度9w是search16w

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

checkpoint = '/root/autodl-tmp/only_dpr/checkpoint7'
checkpoint1 = '/root/autodl-tmp/only_gen/checkpoint9'    

In [13]:
import json

# 打开文件并加载JSON数据
with open('/root/autodl-tmp/rag-end2end-retriever/DIR/squad-v2/ChineseSquad-master/squad_2.0/train-v2.0-zh.json', 'r') as file:
    train = json.load(file)

In [14]:
import pandas as pd


# 初始化空的DataFrame
train_data = pd.DataFrame(columns=['title', 'question', 'answer', 'context'])

# 遍历数据并填充DataFrame

data_to_append = []
for data in train['data']:
    # print (len(data))
    for paragraph in data['paragraphs']:
        context = paragraph['context']
    
        for qa in paragraph['qas']:
            title = data['title']
            question = qa['question']
        
            # 处理多个答案的情况，取第一个答案
            answer = qa['answers'][0]['text'] if qa['answers'] else None
        
        # 将数据添加到列表中
            data_to_append.append({'title': title, 'question': question, 'answer': answer, 'context': context})

# 一次性使用concat追加所有数据
train_data = pd.concat([train_data, pd.DataFrame(data_to_append)], ignore_index=True)

In [15]:
# 在原地删除 'answer' 列中包含 None 的行
train_data.dropna(subset=['answer'], inplace=True)

# 加载模型，记得修改检索条目，知识库路径↑↑

In [50]:
retriever = RagRetriever.from_pretrained(checkpoint, **model_kwargs)                   # here. load models
model = model_class.from_pretrained(checkpoint,  retriever=retriever,**model_kwargs)
model.retriever.init_retrieval()   

# model

 # 推断

In [9]:
final=[]
count=0
def inference_100(q,k=5):     #这里的k仅仅控制检索，并不能控制生成时候检索了多少条

    #首先定义问题问题列表
    
    inputs_dict = model.retriever.question_encoder_tokenizer.batch_encode_plus(
        [q], return_tensors="pt", padding=True, truncation=True
    )
    input_ids=inputs_dict.input_ids
    attention_mask = inputs_dict.attention_mask
    question_hidden_states = model.question_encoder(input_ids,attention_mask)[0]    
    # print(question_hidden_states.shape)

    outputs = model.generate(  # rag_model overwrites generate
            input_ids,
            attention_mask=attention_mask,
            num_beams=3,
            # min_length=args.min_length,
            # min_length=320,
            max_length=500,
            early_stopping=False,
            num_return_sequences=1,
            temperature=0.2,
            bad_words_ids=[[0, 0]],  # BART likes to repeat BOS tokens, dont allow it to generate more than one
        )
    answers = model.retriever.generator_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    retrieved_doc_embeds, doc_ids, docs=model.retriever.retrieve(question_hidden_states.detach().cpu().numpy(),k)
    
    doc = [str(i) + '\n' for i in docs[0]['text']]
    d = ''
    for i in doc:
        d += i
    print('\n问题：\n' + q + '\n\n检索结果：\n' + d + f'\n回答：\n{answers}' )
    # return '\n问题：\n' + q + '\n\n检索：\n' + d + f'\n回答：\n{answers}' 

In [76]:
final=[]
count=0
def inference_100(q,k=5):     #这里的k仅仅控制检索，并不能控制生成时候检索了多少条

    #首先定义问题问题列表
    
    inputs_dict = model.retriever.question_encoder_tokenizer.batch_encode_plus(
        [q], return_tensors="pt", padding=True, truncation=True
    )
    input_ids=inputs_dict.input_ids
    attention_mask = inputs_dict.attention_mask
    question_hidden_states = model.question_encoder(input_ids,attention_mask)[0]    
    # print(question_hidden_states.shape)

    outputs = model.generate(  # rag_model overwrites generate
            input_ids,
            attention_mask=attention_mask,
            num_beams=3,
            # min_length=args.min_length,
            # min_length=320,
            max_length=500,
            early_stopping=False,
            num_return_sequences=1,
            temperature=0.2,
            bad_words_ids=[[0, 0]],  # BART likes to repeat BOS tokens, dont allow it to generate more than one
        )
    answers = model.retriever.generator_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    retrieved_doc_embeds, doc_ids, docs=model.retriever.retrieve(question_hidden_states.detach().cpu().numpy(),k)
    
    doc = [str(i) + '\n' for i in docs[0]['text']]
    d = ''
    for i in doc:
        d += i
    print('\n问题：\n' + q + '\n\n检索结果：\n' + d + f'\n回答：\n{answers}' )
    # return gold(q)

# 验证集

In [16]:
import pandas as pd

In [22]:
def gold(q):
    questions_to_select = [q]
    selected_df = train_data[train_data['question'].isin(questions_to_select)]
    return selected_df 

In [23]:
gold('是谁和盖伊·里奇一起开的酒杯？')

,title,question,answer,context
9710,酒吧,是谁和盖伊·里奇一起开的酒杯？,麦当娜,议会广场上的红狮靠近威斯敏斯特宫，因此被政治记者和议员使用。酒吧配有一个分区钟，当要求议员参...


In [25]:
inference_100('是谁和盖伊·里奇一起开的酒杯？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
是谁和盖伊·里奇一起开的酒杯？

检索结果：
议会广场上的红狮靠近威斯敏斯特宫，因此被政治记者和议员使用。酒吧配有一个分区钟，当要求议员参加投票时，该钟会将议员传回会议厅。“潘趣碗”梅菲尔曾一度由麦当娜和盖伊·里奇共同拥有。厄尔法院的科尔赫内公馆是20世纪50年代以来著名的同性恋酒吧。它吸引了许多著名的赞助人，如弗雷迪·墨丘利、肯尼·埃弗雷特和鲁道夫·努雷耶夫。连环杀手科林·爱尔兰用它来接受害者。
在1975年奥林匹亚先生比赛前几个月，电影制片人乔治·巴特勒和罗伯特·菲奥雷说服施瓦辛格参加比赛，以便拍摄他在健身纪录片《泵铁》中的训练。施瓦辛格只有三个月的时间为比赛做准备，在体重明显减轻之后，他才出现在电影《与杰夫·布里奇斯保持饥饿》中。事实证明，卢·费里尼诺并不是一个威胁，一个比以往更轻的施瓦辛格令人信服地赢得了1975年的奥林匹亚先生。
在意大利，共济会已经与一桩有关宣传的丑闻联系在一起。第2页）。这间小屋是1877年由意大利大东方酒店租用的，作为一个不能去自己的小屋拜访石匠的小屋。在李西奥·盖利的领导下，20世纪70年代末，P2卷入了几乎使梵蒂冈银行破产的金融丑闻。然而，到了这个时候，由于大东方在1976年撤销了它的章程并驱逐了盖利，这个旅馆已经独立和不定期地运作了。
1977年7月，施瓦辛格在威尼斯海滩遇到了他的下一个情人，比佛利山庄美发师的助手苏·莫雷。根据莫雷的说法，这对夫妇的关系是开放的：“当我们都在洛杉矶的时候，我们是忠诚的……但是当他出城的时候，我们可以随心所欲。”1977年8月，施瓦辛格在罗伯特肯尼迪网球锦标赛上认识了玛丽亚·施赖弗，并与两名女子继续保持关系，直到1978年8月，莫雷（知道自己与施赖弗的关系）发出最后通牒。
1929年5月13日至5月16日，约翰逊主持召开了一次全美有组织犯罪人士会议。召集这次会议的人是马塞里亚家族的查尔斯“幸运”卢西亚诺中尉和前芝加哥南区黑帮头目约翰尼“狐狸”托里奥，他们的头目是臭虫和迈耶暴徒迈耶兰斯基和本杰明西格尔，他们被用作会议的肌肉。

回答：
['麦 当 娜']


,title,question,answer,context
9710,酒吧,是谁和盖伊·里奇一起开的酒杯？,麦当娜,议会广场上的红狮靠近威斯敏斯特宫，因此被政治记者和议员使用。酒吧配有一个分区钟，当要求议员参...


In [57]:
inference_100('是谁和盖伊·里奇一起开的酒杯？',5)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
是谁和盖伊·里奇一起开的酒杯？

检索结果：
议会广场上的红狮靠近威斯敏斯特宫，因此被政治记者和议员使用。酒吧配有一个分区钟，当要求议员参加投票时，该钟会将议员传回会议厅。“潘趣碗”梅菲尔曾一度由麦当娜和盖伊·里奇共同拥有。厄尔法院的科尔赫内公馆是20世纪50年代以来著名的同性恋酒吧。它吸引了许多著名的赞助人，如弗雷迪·墨丘利、肯尼·埃弗雷特和鲁道夫·努雷耶夫。连环杀手科林·爱尔兰用它来接受害者。
酒吧通常有传统的名字。一个常见的名字是“格兰比侯爵”。这些酒馆以约翰·曼蒂斯命名，她是格兰比侯爵，约翰·曼蒂斯的儿子，拉特兰第三公爵，18世纪英国军队的一位将军。他非常关心他的部下的福利，在他们退休后，他为他们中的许多人提供了建立酒馆的资金，酒馆后来以他的名字命名。1780年，在乔治三世国王加冕二十周年纪念日之后，所有被授予执照的酒吧都被称为皇家乔治。
柴郡斯大林布里奇镇被认为拥有英国最长和最短的酒吧——老的柴郡第13步枪队客栈和Q客栈。
第二个纽黑文美食名望是路易午餐，它位于皇冠街的一个小砖房里，从1895年开始供应快餐。尽管争论激烈，但该餐厅的创始人路易拉森（Louis Lassen）因发明汉堡和牛排三明治而被美国国会图书馆（Library of Congress）誉为“功臣”。路易的午餐是用1898年的老式铸铁炉子垂直烤汉堡、牛排三明治和热狗，炉子是用烤架做的，这是当地居民路易吉·皮拉戈斯蒂尼在1939年申请的专利，烤肉时炉子会把肉放在适当的位置。
众所周知，伦敦的许多酒吧都曾被名人使用过，但在某些情况下，比如塞缪尔·约翰逊（Samuel Johnson）和叶奥德·柴郡奶酪（Ye Olde Cheshire Cheese）之间的联系，这是一种推测，其依据不过是人们所知此人住在附近。然而，众所周知，查尔斯·狄更斯曾参观过柴郡奶酪、惠特比的前景、老公鸡酒馆和许多其他地方。塞缪尔·佩皮斯也与惠比和公鸡酒馆的前景有关。

回答：
['麦 当 娜']


,title,question,answer,context
9710,酒吧,是谁和盖伊·里奇一起开的酒杯？,麦当娜,议会广场上的红狮靠近威斯敏斯特宫，因此被政治记者和议员使用。酒吧配有一个分区钟，当要求议员参...


In [26]:
inference_100('什么是表达两国间协议的正式文件？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
什么是表达两国间协议的正式文件？

检索结果：
条约是一种官方的、明确的书面协议，各国用来对自己进行法律约束。条约是用文字表达这一协议的正式文件，也是承认双方及其确定关系的仪式的客观结果。
条约制定的一个重要部分是，签署一项条约意味着承认另一方是主权国家，而且正在审议的协定根据国际法是可执行的。因此，各国可以非常谨慎地将一项协议定为一项条约。例如，在美国，州之间的协议是契约，州与联邦政府或政府机构之间的协议是谅解备忘录。
条约是国际法行为体，即主权国家和国际组织根据国际法缔结的协定。条约也可以称为（国际）协定、议定书、公约、公约、公约或换文等条款。不管用什么术语，根据国际法，所有这些形式的协定都是同等考虑的条约，规则是相同的。
目前，国际协议是以执行协议而不是以10:1的比例签署的。尽管行政协议相对容易达成，总统仍然经常选择通过正式的条约程序，而不是行政协议，以便在需要国会通过实施立法或适当资金的事项上，以及那些强加长期、复杂法律义务的协议上，获得国会的支持关于美国。例如，美国、伊朗和其他国家达成的协议不是一项条约。
双边条约是在两个国家或实体之间缔结的。然而，双边条约可能有两个以上的缔约方；例如，考虑瑞士拒绝《欧洲经济区协定》后瑞士与欧洲联盟（欧盟）之间的双边条约。这些条约都有17个缔约方。然而，这些仍然是双边条约，而不是多边条约。缔约方分为两组，瑞士（“一方”）和欧盟及其成员国（“另一方”）。该条约规定了瑞士和欧盟以及各成员国之间的权利和义务，但并未规定欧盟及其成员国之间的任何权利和义务。[需要引用]

回答：
['条 约']


,title,question,answer,context
15706,条约,什么是表达两国间协议的正式文件？,条约,条约是一种官方的、明确的书面协议，各国用来对自己进行法律约束。条约是用文字表达这一协议的正式...


In [58]:
inference_100('什么是表达两国间协议的正式文件？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
什么是表达两国间协议的正式文件？

检索结果：
条约是一种官方的、明确的书面协议，各国用来对自己进行法律约束。条约是用文字表达这一协议的正式文件，也是承认双方及其确定关系的仪式的客观结果。
条约及其修正案通过后，必须遵循法律事务厅适用的联合国正式法律程序，包括签署、批准和生效。
条约制定的一个重要部分是，签署一项条约意味着承认另一方是主权国家，而且正在审议的协定根据国际法是可执行的。因此，各国可以非常谨慎地将一项协议定为一项条约。例如，在美国，州之间的协议是契约，州与联邦政府或政府机构之间的协议是谅解备忘录。
条约是国际法行为体，即主权国家和国际组织根据国际法缔结的协定。条约也可以称为（国际）协定、议定书、公约、公约、公约或换文等条款。不管用什么术语，根据国际法，所有这些形式的协定都是同等考虑的条约，规则是相同的。
目前，国际协议是以执行协议而不是以10:1的比例签署的。尽管行政协议相对容易达成，总统仍然经常选择通过正式的条约程序，而不是行政协议，以便在需要国会通过实施立法或适当资金的事项上，以及那些强加长期、复杂法律义务的协议上，获得国会的支持关于美国。例如，美国、伊朗和其他国家达成的协议不是一项条约。

回答：
['条 约 及 其 确 定 关 系 的 仪 式']


,title,question,answer,context
15706,条约,什么是表达两国间协议的正式文件？,条约,条约是一种官方的、明确的书面协议，各国用来对自己进行法律约束。条约是用文字表达这一协议的正式...


In [27]:
inference_100('印度和巴基斯坦在什么时间内发现了居住现象？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
印度和巴基斯坦在什么时间内发现了居住现象？

检索结果：
印度次大陆的中石器时代之后是新石器时代，大约12000年前末冰期结束后，次大陆发生了更广泛的沉降。9000年前，在印度现代中央邦（Madhya Pradesh）的Bhimbetka岩石避难所，首次确认的半永久性定居点出现。南亚早期新石器时代文化的代表是印度哈里亚纳邦的比洛纳发现（公元前7500年）和巴基斯坦俾路支省的迈赫加尔发现（公元前7000-9000年）。
印度中部纳尔玛达山谷哈特诺拉的直立人遗骸表明，印度可能至少在50万至20万年前的中更新世就有人居住。在印度次大陆西北部发现了由原始人类制作的工具，这些工具可以追溯到两百万年前。该地区的古代历史包括南亚一些最古老的定居点和一些主要文明。次大陆最早的考古遗址是索安河谷旧石器时代的原始人遗址。索尼安遗址发现于锡瓦利克地区，横跨现在的印度、巴基斯坦和尼泊尔。
除了对独立的渴望，印度教和穆斯林之间的紧张关系也在这些年来不断发展。穆斯林一直是印度次大陆的少数民族，印度政府的排他性使他们对独立持谨慎态度；他们倾向于不信任印度的统治，就像他们抵制外国统治一样，尽管甘地以惊人的领导力展示了呼吁两个群体团结一致。英国在第二次世界大战中极度衰弱，承诺他们将离开并参加临时政府的组建。英属印度领土在1947年被划分为印度联盟和巴基斯坦领土后获得独立。继有争议的旁遮普和孟加拉分治前，这些省份的锡克教、印度教徒和穆斯林爆发暴乱，并蔓延到印度其他几个地区，造成约50万人死亡。此外，这一时期发生了有史以来最大规模的大规模移民，共有1200万印度教徒、锡克教徒和穆斯林在印度和巴基斯坦这两个新创建的国家之间流动（分别于1947年8月15日和14日获得独立）。1971年，原东巴基斯坦和东孟加拉邦的孟加拉国脱离巴基斯坦。
人们普遍认为，架构的成功是一个不断尝试和错误的过程的产物，随着这个过程的结果越来越令人满意，尝试的次数越来越少，复制的次数也越来越多。所谓的乡土建筑在世界许多地方仍在继续生产。事实上，白话建筑构成了人们每天所经历的大部分建筑世界。早期的人类住区大多是农村。由于生产过剩，经济开始扩张，导致城市化，从而形成了城市地区，在某些情况下增长和发展非常迅速，例如安纳托利亚的圣阿塔尔·胡克和现代巴基斯坦印度河流域文明的莫亨乔·达罗。
成熟的印度文明在

,title,question,answer,context
57492,印度历史,印度和巴基斯坦在什么时间内发现了居住现象？,公元前7000-9000年,印度次大陆的中石器时代之后是新石器时代，大约12000年前末冰期结束后，次大陆发生了更广泛的...


In [59]:
inference_100('印度和巴基斯坦在什么时间内发现了居住现象？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
印度和巴基斯坦在什么时间内发现了居住现象？

检索结果：
印度次大陆的中石器时代之后是新石器时代，大约12000年前末冰期结束后，次大陆发生了更广泛的沉降。9000年前，在印度现代中央邦（Madhya Pradesh）的Bhimbetka岩石避难所，首次确认的半永久性定居点出现。南亚早期新石器时代文化的代表是印度哈里亚纳邦的比洛纳发现（公元前7500年）和巴基斯坦俾路支省的迈赫加尔发现（公元前7000-9000年）。
印度中部纳尔玛达山谷哈特诺拉的直立人遗骸表明，印度可能至少在50万至20万年前的中更新世就有人居住。在印度次大陆西北部发现了由原始人类制作的工具，这些工具可以追溯到两百万年前。该地区的古代历史包括南亚一些最古老的定居点和一些主要文明。次大陆最早的考古遗址是索安河谷旧石器时代的原始人遗址。索尼安遗址发现于锡瓦利克地区，横跨现在的印度、巴基斯坦和尼泊尔。
19世纪最重要的事件之一是印度民族主义的兴起，导致印度人首先寻求“自治”，然后寻求“完全独立”。然而，历史学家对其兴起的原因众说纷纭。可能的原因包括“印度人民与英国利益的利益冲突”、“种族歧视”、“印度过去的启示”、“不同地区新社会群体的相互联系”，以及印度人与“欧洲教育”的密切接触。
罗米拉·塔帕尔指出，印度历史分为印度教-穆斯林-英国时期，过于强调“统治王朝和外国入侵”，忽视了往往表现出强烈延续性的社会经济历史。分为古代中世纪和现代时期，忽视了穆斯林征服逐渐发生的事实，在这期间，许多事情来了又走了，而南方从来没有完全征服。根据Thapar的说法，一个周期也可以建立在“重大的社会和经济变化”的基础上，这些变化与统治权力的变化没有严格的关系。[注1]
成熟的印度文明在公元前2600年到1900年间蓬勃发展，标志着次大陆城市文明的开始。这些文明包括现代印度的城市中心，如多拉维拉、卡利班甘、罗帕尔、拉希加尔希和洛萨，以及现代巴基斯坦的哈拉帕、甘尼瓦拉和莫亨乔达罗。这个文明的城市以其砖砌、路边排水系统和多层房屋而闻名，并被认为有某种市政组织。

回答：
['公 元 前 7000 - 9000 年']


,title,question,answer,context
57492,印度历史,印度和巴基斯坦在什么时间内发现了居住现象？,公元前7000-9000年,印度次大陆的中石器时代之后是新石器时代，大约12000年前末冰期结束后，次大陆发生了更广泛的...


In [28]:
inference_100('中日两国在哪一年陷入僵局？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
中日两国在哪一年陷入僵局？

检索结果：
冷战期间，日本的亚洲大国和英国、法国、西德的欧洲大国重建了各自的经济。法国和联合王国至今仍保持着技术先进的武装部队，拥有投射力量的能力，并保持着庞大的国防预算。然而，随着冷战的继续，当局开始质疑法国和英国是否能保持其长期以来的大国地位。中国作为世界上人口最多的国家，已经慢慢上升到大国地位，战后经济和军事实力都有了较大增长。1949年以后，中华民国开始失去其他列强对中华人民共和国唯一合法政府的承认。后来，在1971年，中国失去了联合国安理会常任理事国席位。
第二次世界大战后，欧洲被非正式地划分为西方和苏联的势力范围。西欧后来加入北大西洋公约组织（北约），东欧加入华沙公约。权力从西欧和大英帝国转移到了美国和苏联这两个新的超级大国。这两个对手稍后将在冷战中对峙。在亚洲，日本的失败导致了民主化。中国内战一直持续到战后，最终建立了中华人民共和国。欧洲列强的前殖民地开始了他们的独立之路。
一项原本有效并经商定的条约可能作为一项具有约束力的国际协定而被否决，原因有好几个，其中大多数涉及在条约形成时产生的问题。[需要引证]例如，1905年、1907年和1910年的日韩系列条约遭到抗议；1965年的《日韩基本关系条约》确认这些条约“已经无效”。
第二次中日战争期间，日本帝国和美国之间的紧张关系加剧；泛美事件和南京大屠杀等事件使美国舆论转而反对日本。1940年至1941年，随着法属印度支那被占领，加上中国战争的持续，美国对日本实施了战略物资禁运，如废金属和石油，这些都是战争所必需的。日本人面临的选择要么是从中国撤军，丢脸，要么是在资源丰富、欧洲控制的东南亚殖民地，特别是英属马来亚和荷兰东印度群岛（现代印度尼西亚）攫取和获取新的原材料来源。1940年，日本帝国与纳粹德国和法西斯意大利签订了三方条约。
1884年，首尔的亲日韩国人领导了加普辛政变。中国为镇压起义进行干预后，中日关系紧张。日本首相ItōHirobumi和李鸿章签署了《天津公约》，一项同时撤军的协议，但是1895年的第一次中日战争是一次军事羞辱。《马关条约》承认朝鲜独立，将台湾和澎湖割让给日本。这些条款可能更为严厉，但当日本公民攻击并打伤李鸿章时，国际社会的强烈抗议使日本人感到羞耻，不得不修改这些条款。最初的协议规定把辽东半岛割让给日本，但俄罗斯

,title,question,answer,context
77456,太平洋战争,中日两国在哪一年陷入僵局？,1941,战争进入了一个新阶段，日本在睢县战役和长沙第一次战役中的空前失败。这些胜利之后，中国国民党军...


In [60]:
inference_100('中日两国在哪一年陷入僵局？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
中日两国在哪一年陷入僵局？

检索结果：
1940年9月，日本决定夺取当时由维希法国控制的印度支那，切断中国唯一对外的陆地线。日军违背了与维希政府的协议，爆发了战斗，以日军的胜利告终。9月27日，日本与德国和意大利签署军事同盟，成为三个轴心国之一。实际上，直到1944年，日本和德国之间几乎没有协调，那时美国正在破译他们的秘密外交信件。
冷战期间，日本的亚洲大国和英国、法国、西德的欧洲大国重建了各自的经济。法国和联合王国至今仍保持着技术先进的武装部队，拥有投射力量的能力，并保持着庞大的国防预算。然而，随着冷战的继续，当局开始质疑法国和英国是否能保持其长期以来的大国地位。中国作为世界上人口最多的国家，已经慢慢上升到大国地位，战后经济和军事实力都有了较大增长。1949年以后，中华民国开始失去其他列强对中华人民共和国唯一合法政府的承认。后来，在1971年，中国失去了联合国安理会常任理事国席位。
日本赞助了几个傀儡政府，其中一个由汪精卫领导。然而，它对中国人民的残暴政策、不向这些政权让出任何实际权力的政策、以及支持几个敌对政府的政策，都未能使它们成为蒋介石领导的民族主义政府的可行替代品。1941年1月，中国共产党和民族主义武装在敌后争夺领土控制权的冲突达到高潮，双方的合作实际上结束了。
在朝鲜战争的剩余时间里，联合国军司令部和PVA进行了战斗，但几乎没有交换领土；僵局依然存在。对朝鲜的大规模轰炸仍在继续，1951年7月10日在开城开始了旷日持久的停战谈判。中方方面，周恩来主持和谈，李克农、乔光华领导谈判小组。在交战双方谈判的同时，战斗仍在继续；联合国司令部的目标是夺回整个韩国，避免失去领土。人民军和人民军试图进行类似的行动，后来进行了军事和心理行动，以考验联合国司令部继续战争的决心。
第二次世界大战后，欧洲被非正式地划分为西方和苏联的势力范围。西欧后来加入北大西洋公约组织（北约），东欧加入华沙公约。权力从西欧和大英帝国转移到了美国和苏联这两个新的超级大国。这两个对手稍后将在冷战中对峙。在亚洲，日本的失败导致了民主化。中国内战一直持续到战后，最终建立了中华人民共和国。欧洲列强的前殖民地开始了他们的独立之路。

回答：
['1940']


,title,question,answer,context
77456,太平洋战争,中日两国在哪一年陷入僵局？,1941,战争进入了一个新阶段，日本在睢县战役和长沙第一次战役中的空前失败。这些胜利之后，中国国民党军...


In [29]:
inference_100('瑞士的葡萄园是什么时候开始种植的？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
瑞士的葡萄园是什么时候开始种植的？

检索结果：
16世纪中叶，“植物园”在意大利多所大学建立，1545年的帕多瓦植物园通常被认为是第一个仍在原址的植物园。这些园林延续了早期“物理园林”的实用价值，通常与寺院相联系，在寺院中种植植物用于医疗。他们支持植物学作为一门学术学科的发展。他们就花园里种植的植物进行了讲座，并演示了它们的医疗用途。植物园在北欧出现得较晚；1621年英国第一个植物园是牛津大学植物园。在这一时期，植物学始终坚定地服从于医学。
植物生态学的学科是19世纪末由植物学家如尤金斯温姆（Eugenius Warming）开创的，他提出了植物形成群落的假设，他的导师和继任者克里斯滕C.Raunkiær（Christen C.Raunkiær）描述植物生命形式的系统至今仍在使用。亨利·钱德勒·考尔斯、亚瑟·坦斯利和弗雷德里克·克莱门茨提出了温带阔叶林等植物群落的组成随生态演替过程而变化的概念。克莱门茨认为顶极植被是一个环境能够支持的最复杂的植被，坦斯利将生态系统的概念引入生物学。尼古拉·瓦维洛夫（1887-1943）在阿方斯·德·坎多利（Alphonse de Candolle）早期大量工作的基础上，对经济植物的生物地理学、起源中心和进化史进行了描述。
植物学起源于史前的草药学，早期人类致力于鉴定食用植物、药用植物和有毒植物，后来又开始种植，使之成为最古老的科学分支之一。中世纪的物理花园，通常附属于修道院，包含医学上重要的植物。他们是第一个附属于大学的植物园的前身，始建于15世纪40年代。其中最早的是帕多瓦植物园。这些花园促进了对植物的学术研究。对他们的藏品进行分类和描述的努力是植物分类学的开端，并导致了1753年卡尔林奈的二项式系统至今仍在使用。
瑞士最受欢迎的酒精饮料是葡萄酒。瑞士因种植的葡萄品种而著名，这是因为它的土壤、空气、海拔和光线的特殊混合，在风土条件上有很大的差异。瑞士葡萄酒主要产于瓦莱、瓦德（拉沃）、日内瓦和提契诺，其中白葡萄酒占少数。从罗马时代起，瑞士就开始种植葡萄园，尽管可以找到一些更古老的葡萄园。最广泛的品种是夏塞拉（在瓦莱被称为芬丹）和黑比诺。梅洛是提契诺生产的主要品种。
驯化葡萄的种植始于6000-8000年前的近东地区。酵母是最早驯化的微生物之一，天然存在于葡萄皮上，导致了葡萄酒等酒精饮料的革新

,title,question,answer,context
46549,瑞士,瑞士的葡萄园是什么时候开始种植的？,罗马时代,瑞士最受欢迎的酒精饮料是葡萄酒。瑞士因种植的葡萄品种而著名，这是因为它的土壤、空气、海拔和光...


In [30]:
inference_100('炎症是由什么引起的？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
炎症是由什么引起的？

检索结果：
炎症是免疫系统对感染的最早反应之一。炎症的症状是发红，肿胀，发热和疼痛，这是由流入组织的血液增加引起的。类花生酸和细胞因子产生炎症，它们由受伤或感染的细胞释放。类花生酸包括产生发热和与炎症相关的血管扩张的前列腺素，以及吸引某些白细胞（白细胞）的白三烯。常见的细胞因子包括负责白细胞之间通讯的白介素；生长因子和细胞毒性因子也可能被释放。这些细胞因子和其他化学物质将免疫细胞募集到感染部位，并在病原体去除后促进受损组织的愈合。
相反，在清醒期，分化的效应细胞，如细胞毒性自然杀伤细胞和CTLs（细胞毒性T淋巴细胞），峰，以引起对任何侵入性病原体的有效反应。同样，在清醒的活动时间里，消炎分子（例如皮质醇和儿茶酚胺）也会达到峰值。关于为什么将促炎状态保留给睡眠时间有两种理论。首先，如果炎症在唤醒时间发生，将会引起严重的认知和身体损伤。其次，由于褪黑激素的存在，炎症可能在睡眠时间发生。炎症会引起大量的氧化应激，并且在睡眠时间内褪黑激素的存在可以积极地抵消这段时间内自由基的产生。
根据沃尔特·格拉泽的说法，对营养的研究可能始于公元前6世纪。在中国，气的概念发展起来，一种类似于西欧后来称之为气的精神或“风”。在中国、印度、马来亚和波斯，食物分为“热”（例如肉、血、姜和热香料）和“冷”（绿色蔬菜）。幽默可能是中国第一个和齐一起发展起来的。何医生总结说，疾病是由元素缺乏引起的（吴星：火、水、土、木、金），他把疾病和规定的饮食分类。大约在意大利的同一时间，克罗顿（希腊）的阿尔克梅昂（Alcmaeon）写下了进进出出平衡的重要性，并警告说，失衡将导致以肥胖或消瘦为特征的疾病。
免疫学是一门检查免疫系统结构和功能的科学。它源自医学和对疾病免疫原因的早期研究。最早提到豁免权的是公元前430年的雅典瘟疫。修昔底德指出，从以前的疾病发作中恢复过来的人可以照料病人，而无需第二次患病。在18世纪，Pierre-Louis Moreau de Maupertuis用蝎子毒液进行了实验，观察到某些狗和老鼠对这种毒液免疫。路易斯·巴斯德（Louis Pasteur）后来在接种疫苗的发展和他提出的疾病病原学理论中利用了这种和其他获得性免疫的观察结果。巴斯德的理论与当代疾病理论（例如opposition气理论）直接相反。直到罗伯特·科赫（R

,title,question,answer,context
59972,哮喘,炎症是由什么引起的？,气道高反应性,虽然哮喘是一种公认的疾病，但没有一个普遍同意的定义。全球哮喘倡议将其定义为“一种气道慢性炎症...


In [61]:
inference_100('炎症是由什么引起的？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
炎症是由什么引起的？

检索结果：
炎症是免疫系统对感染的最早反应之一。炎症的症状是发红，肿胀，发热和疼痛，这是由流入组织的血液增加引起的。类花生酸和细胞因子产生炎症，它们由受伤或感染的细胞释放。类花生酸包括产生发热和与炎症相关的血管扩张的前列腺素，以及吸引某些白细胞（白细胞）的白三烯。常见的细胞因子包括负责白细胞之间通讯的白介素；生长因子和细胞毒性因子也可能被释放。这些细胞因子和其他化学物质将免疫细胞募集到感染部位，并在病原体去除后促进受损组织的愈合。
如果细菌与其他有机体形成寄生联系，它们就被归类为病原体。病原菌是导致人类死亡和疾病的主要原因，可引起破伤风、伤寒、白喉、梅毒、霍乱、食源性疾病、麻风病和结核病等感染。一种已知的医学疾病的致病原因可能在多年后才被发现，如幽门螺杆菌和消化性溃疡病。细菌性疾病在农业中也很重要，细菌会导致植物的叶斑病、火疫病和枯萎病，以及农场动物的约翰氏病、乳腺炎、沙门氏菌和炭疽病。
炎症性腹泻发生在粘膜衬里或刷缘受损时，导致富含蛋白质的液体被动流失，并降低吸收这些流失液体的能力。其他三种腹泻的特征[需要澄清]都可以在这种腹泻中找到。它可能是由细菌感染、病毒感染、寄生虫感染或自身免疫问题（如炎症性肠病）引起的。它也可能由结核病、结肠癌和肠炎引起。[需要引用]
相反，在清醒期，分化的效应细胞，如细胞毒性自然杀伤细胞和CTLs（细胞毒性T淋巴细胞），峰，以引起对任何侵入性病原体的有效反应。同样，在清醒的活动时间里，消炎分子（例如皮质醇和儿茶酚胺）也会达到峰值。关于为什么将促炎状态保留给睡眠时间有两种理论。首先，如果炎症在唤醒时间发生，将会引起严重的认知和身体损伤。其次，由于褪黑激素的存在，炎症可能在睡眠时间发生。炎症会引起大量的氧化应激，并且在睡眠时间内褪黑激素的存在可以积极地抵消这段时间内自由基的产生。
中性粒细胞和巨噬细胞是吞噬细胞，它们在体内传播以追寻入侵的病原体。中性粒细胞通常存在于血液中，是最丰富的吞噬细胞，通常占循环白细胞总数的50%至60%。在炎症的急性期，特别是由于细菌感染，中性粒细胞以称为趋化性的过程向炎症部位迁移，通常是最早到达感染现场的细胞。巨噬细胞是存在于组织内的多功能细胞，产生多种化学物质，包括酶、补体蛋白和调节因子，如白细胞介素1。巨噬细胞还起到清除剂的作用，清除掉破旧的细胞

,title,question,answer,context
59972,哮喘,炎症是由什么引起的？,气道高反应性,虽然哮喘是一种公认的疾病，但没有一个普遍同意的定义。全球哮喘倡议将其定义为“一种气道慢性炎症...


In [34]:
inference_100('波士顿大约有多少人在爆炸中受伤？',5)    #这个有现象

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
波士顿大约有多少人在爆炸中受伤？

检索结果：
2010年，波士顿估计有617594名居民（密度为12200人/平方英里，或4700人/平方公里）居住在272481套住房中，比2000年人口增长了5%。这座城市是美国人口第三稠密的大城市，拥有50多万居民。大约120万人在工作时间可能在波士顿境内，特别活动期间可能多达200万人。这种人口波动是由成千上万的郊区居民前往城市工作、教育、医疗和特殊活动造成的。
2013年4月15日，两名车臣伊斯兰兄弟在波士顿马拉松终点附近引爆两枚炸弹，造成3人死亡，约264人受伤。
波士顿家庭收入中值为51739美元，而家庭收入中值为61035美元。全年全职男性员工的收入中值为52544美元，全年全职女性员工的收入中值为46540美元。这个城市的人均收入是33158美元。21.4%的人口和16.0%的家庭生活在贫困线以下。在总人口中，18岁以下人口占28.8%，65岁以上人口占20.4%。
1886年8月31日，查尔斯顿几乎被地震摧毁。冲击估计有7的矩量和最大的MelCALI强度X（极值）。它被感觉到北至波士顿，西北至芝加哥和密尔沃基，西至新奥尔良，南至古巴，东至百慕大。它毁坏了查尔斯顿的2000座建筑物，造成了600万美元的损失（2006年为1.33亿美元），而当时该市所有建筑物的价值约为2400万美元（2006年为5.31亿美元）。
波士顿的高校对地区经济有着重要的影响。波士顿吸引了来自世界各地的35万多名大学生，他们每年为波士顿的经济贡献超过48亿美元。该地区的学校是主要的雇主，吸引工业到城市和周边地区。波士顿是许多科技公司的所在地，也是生物技术中心，米尔肯研究所（Milken Institute）将波士顿评为美国顶尖的生命科学集群。波士顿每年从美国所有城市的国立卫生研究院获得的绝对金额最高。

回答：
['264']


,title,question,answer,context


In [35]:
inference_100('英国官邸的首相是什么？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
英国官邸的首相是什么？

检索结果：
英国和联合王国的君主有大臣，他们特别信任他们，他们被视为政府首脑。例如亨利八世时期的托马斯·克伦威尔，伊丽莎白一世时期的威廉·塞西尔，伯格利勋爵，查理二世时期的克拉伦登，安妮女王时期的戈多芬。这些部长担任各种正式职务，但通常被称为“部长”、“首席部长”、“第一部长”和最后的“总理”。
联合王国的宪法没有修改，基本上没有成文，没有提到首相。虽然它事实上已经存在了几个世纪，但在官方的国家文件中首次提到，直到二十世纪的第一个十年才发生。因此，它常被称为“不存在”，事实上有几个例子表明议会是这样的。首相在内阁中的职位完全是靠另一个职位，要么是第一任财政大臣（委任职位），要么是财政大臣（最后一位是1905年的鲍尔福）。
首相一词在我们所知的意义上起源于18世纪的英国，当时国会议员轻蔑地用这个头衔来指罗伯特·沃尔波尔爵士。随着时间的推移，这个头衔成为了敬语，在21世纪仍然如此。
在非英联邦国家，首相有权享有总统般的尊称。在一些英联邦国家，总理和前总理因其职位而被称为“尊敬的人”，例如加拿大总理。在联合王国，首相和前首相似乎也被称为尊贵的人，但这并不是由于他们作为政府首脑的地位，而是作为女王陛下最尊贵的枢密院现任成员的特权。
英文版的公约将几乎所有国家的政府首脑称为“总理”（有时改为相当于总理的任期），而不论其各自国家的政府首脑的正确头衔如何。这一规则的少数例外是德国和奥地利，它们的政府首脑头衔几乎总是被翻译成总理；摩纳哥的政府首脑被称为国务部长；梵蒂冈城的政府首脑被称为国务卿。就爱尔兰而言，讲英语的人有时把政府首脑称为Taoiseach。一个突出的例子是伊朗总统，他实际上不是国家元首，而是伊朗政府首脑。他在波斯语和英语中都被称为“总统”。

回答：
['丘 吉 尔']


,title,question,answer,context
55681,伦敦,英国官邸的首相是什么？,唐宁街10号,伦敦是联合王国政府所在地。许多政府部门都设在威斯敏斯特宫附近，特别是沿着白厅，包括唐宁街10...


In [36]:
inference_100('9月27日肖邦在哪个城市演出？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
9月27日肖邦在哪个城市演出？

检索结果：
最近有几部电影在纽黑文拍摄，包括蒙娜丽莎·斯迈尔（2003年）、茱莉亚·罗伯茨（2007年）、乌玛·瑟曼（Uma Thurman）、印第安纳·琼斯（Indiana Jones）和史蒂文·斯皮尔伯格（Steven Spielberg）执导、哈里森·福特（Harrison Ford）、凯特·布兰切特（Cate Blanchett）和希亚·拉博夫（Shia LaBeouf）主演的《水晶骷髅王国》（2008年）。《水晶骷髅》的拍摄过程中，在纽黑文的街道上进行了大量的追逐。几条市中心街道禁止交通，并进行了一次“改造”，使之看起来像电影拍摄时的1957年的街道。500名当地人被选为这部电影的临时演员。在《人人好》（2009）中，罗伯特·德尼罗在丹佛火车站有一次亲密的邂逅；这一幕是在纽黑文的联合火车站拍摄的。
许多以表演和视觉艺术为中心的年度活动吸引游客到安娜堡。一个这样的活动是安阿伯艺术博览会，一套四个并行的陪审团集市举行的市中心街道。定于7月第三周的星期三至星期六举行的博览会吸引了50万游客。另一个是在3月第三周举办的安娜堡电影节，每年收到来自40多个国家的2500多份投稿，是美国屈指可数的奥斯卡颁奖典礼之一。
可能第一次尝试虚构肖邦的生活是一个幻想的歌剧版本的一些事件。肖邦是由奥雷菲斯写的，1901年在米兰制作。所有的音乐都源于肖邦的音乐。
在沙泉，一个叫做“发现乐园”的露天竞技场是俄克拉何马州音乐剧的官方演出总部！里奇·邦德，俄克拉何马州麦克莱斯特人，曾主演过俄克拉何马州的百老汇和国际巡回演出！在2600多场演出中扮演“卷发麦克莱恩”的角色。1953年，他和俄克拉荷马州一起被选中！在哥伦比亚广播公司的综合电视节目中播出。邦德在《成为俄克拉荷马州之歌》的标题歌中发挥了重要作用，并在纪念该音乐剧50周年的美国邮票上出现。从历史上看，该州已经产生了诸如塔尔萨之声和西部秋千等音乐风格，并在塔尔萨凯恩的舞厅得到普及。这座被称为“卡内基西部摇摆厅”的建筑在20世纪30年代曾是鲍勃·威尔斯和德州花花公子的表演总部。斯蒂尔沃特被称为红土音乐的震中，其中最著名的支持者是已故的鲍勃·柴尔德。
第五大道剧院建于1926年，上演百老汇风格的音乐剧，既有当地的天才，也有国际明星。西雅图有“大约100家”的戏剧制作

,title,question,answer,context
598,肖邦,9月27日肖邦在哪个城市演出？,格拉斯哥,夏末，简·斯特林邀请他访问苏格兰，他住在爱丁堡附近的考尔德宫和伦弗雷夫斯郡的约翰斯顿城堡，这...


In [62]:
inference_100('9月27日肖邦在哪个城市演出？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
9月27日肖邦在哪个城市演出？

检索结果：
1839年在巴黎举行的男高音阿道夫·努里特（Adolphe Nourrit）葬礼上，肖邦罕见地出现在风琴上，演奏了弗兰兹·舒伯特（Franz Schubert）的《说谎的手势》（lied Die Gestirne）的抄本。1840年7月26日，肖邦和桑德出席了为纪念7月革命10周年而创作的柏辽兹大交响曲《欢乐合唱团》的彩排。据报道，肖邦对这部作品并不感兴趣。
11月底，肖邦回到巴黎。他不停地生病过冬，但偶尔上几节课，朋友们也来拜访他，包括德拉克罗瓦和弗朗索姆。他偶尔为朋友们演奏或伴奏德尔菲娜·波托卡的歌声。1849年夏天，他的朋友们在市中心的切洛特为他找到了一套公寓，租金由一位崇拜者奥布雷斯科夫公主秘密补贴。1849年6月，珍妮·林德拜访了他。
1848年11月16日，肖邦在伦敦市政厅的一个音乐会讲台上最后一次公开露面，当时他以最后的爱国姿态为波兰难民演奏。这时他病得很重，体重不足99磅。不到45公斤），他的医生知道他的病已经到了晚期。
在诺汉的夏天，尤其是1839-1833年，肖邦发现了安静而富有成效的日子，在这期间他创作了许多作品，包括他在A大调的波兰舞曲，作品。53.在诺汉的来访者中有德拉克罗瓦和女中音保琳·维亚多，肖邦曾为他们提供钢琴技巧和作曲方面的建议。德拉克罗瓦在1842年6月7日的一封信中叙述了在诺汉的住宿情况：
在伦敦，肖邦住在多佛街，布罗德伍德公司为他提供了一架大钢琴。在他5月15日在斯塔福德宫举行的第一次订婚仪式上，观众包括维多利亚女王和阿尔伯特亲王。王子本人就是一位天才的音乐家，他靠近键盘观看肖邦的演奏技巧。布罗德伍德还为他安排了音乐会；参加音乐会的有萨克雷和歌手珍妮·林德。肖邦在钢琴课上也受到追捧，他每小时收取一几内亚（以目前的英国货币计算为1.05英镑）的高额费用，在私人演奏会上收取20几内亚的费用。在7月7日的一场音乐会上，他与维亚尔多分享了这个平台，维亚尔多演唱了他的一些玛祖卡曲，并将其改编成西班牙文。

回答：
['伦 敦']


,title,question,answer,context
598,肖邦,9月27日肖邦在哪个城市演出？,格拉斯哥,夏末，简·斯特林邀请他访问苏格兰，他住在爱丁堡附近的考尔德宫和伦弗雷夫斯郡的约翰斯顿城堡，这...


In [37]:
inference_100('明朝末代皇帝为什么自杀？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
明朝末代皇帝为什么自杀？

检索结果：
明朝政府官员互相斗争，反对财政崩溃，反对一系列农民起义。他们无法利用满族继承纠纷和设置未成年人为皇帝。1644年4月，北京的首都被李自成领导的反叛力量联盟洗劫，李自成是明朝的一位前小官员，他建立了一个短暂的顺朝。明朝最后一位统治者，崇祯皇帝，在城市倒塌时自杀，标志着王朝的正式结束。
在1253年征服大理之后，前执政的段王朝被任命为总督，元，明，清时期的政府均认可其为皇室官员，主要分布在云南省。但是，元朝的继承是一个棘手的问题，后来引起了许多冲突和内部斗争。这最早是在忽必烈统治结束时出现的。忽必烈最初将其长子甄真命名为王储，但他于1285年在忽必烈之前去世。因此，在1294年至1307年间，镇津的第三个儿子在其母亲科克津（Kökejin）和部长巴彦（Bayan）的支持下继承了王位，并统治着天武汗（TemürKhan）或成宗皇帝。TemürKhan决定维持并继续他祖父开始的许多工作。他还与蒙古西部的汗国和越南等邻国和睦相处，越南承认他的宗主国主权，并为之献礼了几十年。但是，元朝的腐败始于泰米尔汗（TemürKhan）统治时期。
新联军于6月6日攻占北京。顺治皇帝于10月30日被封为“天子”。满族通过击败叛军李自成，将自己定位为明朝皇帝的政治继承人，通过为崇祯皇帝举行正式葬礼，完成了象征性的过渡。然而，征服中国其他地区的过程，又花了17年的时间与明朝的忠臣、伪君子和叛军作战。最后一个明朝冒充者桂王子向缅甸国王寻求庇护，但被交给吴三桂指挥的清朝远征军，吴三桂将他带回云南省，并于1662年初被处决。
不稳定困扰着忽必烈汗统治初期。Ogedei的孙子Kaidu拒绝服从Kublai，并威胁到Kublai领土的西部边界。敌对但衰弱的宋朝仍然是南方的障碍。1259年，忽必烈通过将人质王子原宗作为朝鲜的统治者，使其成为蒙古的朝贡国，从而确保了东北边界的安全。忽必烈也受到国内动荡的威胁。厉害的女son李潭在1262年煽动了对蒙古统治的起义。在成功镇压叛乱之后，忽必烈在法院遏制了汉族顾问的影响。他担心他对中国官员的依赖使他容易遭受未来叛乱和对宋人的叛逃。
从1340年代后期开始，农村地区的人们经常遭受自然灾害的困扰，例如干旱，洪水和由此造成的饥荒，政府缺乏有效的政策导致失去了民众的支持。1351年，红色头巾叛乱

,title,question,answer,context
73990,现代史,明朝末代皇帝为什么自杀？,北京被李自成的农民起义军占领,清朝（1644-1911）是在明朝（中国最后一个汉朝）垮台后由满族人建立的。满族以前被称为女...


In [63]:
inference_100('明朝末代皇帝为什么自杀？',5)

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
明朝末代皇帝为什么自杀？

检索结果：
明朝政府官员互相斗争，反对财政崩溃，反对一系列农民起义。他们无法利用满族继承纠纷和设置未成年人为皇帝。1644年4月，北京的首都被李自成领导的反叛力量联盟洗劫，李自成是明朝的一位前小官员，他建立了一个短暂的顺朝。明朝最后一位统治者，崇祯皇帝，在城市倒塌时自杀，标志着王朝的正式结束。
迁都北京两个多世纪后，南京又一次注定要成为明朝皇帝的都城。1644年春，北京沦陷到李自成的反叛势力手中，接着又沦陷到满清领导的清朝，1644年6月，明太子朱有松在南京即位，成为红光皇帝。他的短暂统治被后来的历史学家描述为所谓南明的第一个统治时期。
在他篡夺王位的Jianwen Emperor（R. 1398 - 1402）中，Yongle Emperor得到了僧侣姚光晓的帮助，和他的父亲Hongwu Emperor一样，Yongle Emperor“对佛教有好感”，Rossabi说。1403年3月10日，永乐皇帝邀请第五世噶玛巴喇嘛（1384-1415）德辛·谢帕（Deshin Shekpa）到庭，尽管第四世噶玛巴拒绝了洪武皇帝的邀请。16世纪的藏文译本保存了永乐皇帝的信，亚洲研究协会指出，这封信是对噶玛巴的礼貌和赞美。邀请函上写着，
随后在1637年创建了前两个汉旗（1642年增加到八个）。这些军事改革，使洪太基在1640年至1642年的一系列战役中，为松山和锦州的领土，响亮地击败了明军。这场最后的胜利导致许多明朝最顽强的军队投降，袁崇焕死在崇祯皇帝手中（他认为袁崇焕背叛了他），剩下的明朝军队全部永久撤出长城以北。
新联军于6月6日攻占北京。顺治皇帝于10月30日被封为“天子”。满族通过击败叛军李自成，将自己定位为明朝皇帝的政治继承人，通过为崇祯皇帝举行正式葬礼，完成了象征性的过渡。然而，征服中国其他地区的过程，又花了17年的时间与明朝的忠臣、伪君子和叛军作战。最后一个明朝冒充者桂王子向缅甸国王寻求庇护，但被交给吴三桂指挥的清朝远征军，吴三桂将他带回云南省，并于1662年初被处决。

回答：
['崇 祯 皇 帝']


,title,question,answer,context
73990,现代史,明朝末代皇帝为什么自杀？,北京被李自成的农民起义军占领,清朝（1644-1911）是在明朝（中国最后一个汉朝）垮台后由满族人建立的。满族以前被称为女...


In [39]:
inference_100('为什么国王会在这期间提高税率？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
为什么国王会在这期间提高税率？

检索结果：
中世纪晚期，强大的、以皇室为基础的民族国家在整个欧洲兴起，特别是在英格兰、法国和伊比利亚半岛的基督教王国：阿拉贡、卡斯蒂利亚和葡萄牙。这一时期的长期冲突加强了皇室对其王国的控制，对农民极为不利。国王们从战争中获益，战争扩大了王室的立法，增加了他们直接控制的土地。为战争买单要求税收方法变得更加有效和高效，税率经常提高。获得纳税人同意的要求允许英国议会和法国房地产总公司等代表机构获得权力和权威。
罗马帝国在公元2世纪达到了它最大的领土范围；随后的两个世纪见证了罗马对其边远领土控制的缓慢衰落。包括通货膨胀在内的经济问题和边境地区的外部压力加在一起，使得3世纪的政治不稳定，皇帝登基后，很快就被新的篡位者取代。军事开支在3世纪稳步增加，主要是为了应对3世纪中叶苏醒的与萨珊王朝波斯的战争。军队规模扩大了一倍，骑兵和小部队取代军团成为主要战术单位。对税收的需求导致了税收的增加和居里阶级或土地所有阶级的数量的减少，以及愿意承担在家乡任职的人数的减少。中央政府需要更多的官僚来处理军队的需要，这导致平民抱怨说，帝国的收税人比纳税人多。
白酒是另一个盈利的私人产业，在公元前98年被中央政府国有化。然而，这一规定在公元前81年被废除，对那些私下交易的人，每0.2升（0.05加仑）征收两枚硬币的财产税。公元前110年，武帝还通过以低于商人要求的价格出售政府储备的粮食来消除投机行为，从而干预了有利可图的粮食贸易。除了明朝在公元68年设立了一个短期的调价和稳定办公室，中央政府的价格管制条例在东汉时期基本上没有。
随着金融的扩张，贸易权越来越受到商业精英的谨慎保护。城镇看到了行会日益强大的力量，而在国家一级，特殊公司将被授予对特定行业的垄断权，如英国羊毛主食。这些发展的受益者将积累巨大的财富。德国的富格尔家族、意大利的美第奇家族、英国的德拉波尔家族以及法国的雅克·科厄等个人将帮助资助国王的战争，并在这一过程中取得巨大的政治影响力。
班超（公元102年）征募了库山帝国的援助，占领了现代印度、巴基斯坦、阿富汗和塔吉克斯坦地区，以制服喀什及其盟友索格迪亚纳。当库山统治者维玛·卡德皮斯（Vima Kadphis）提出要求时（r.c.90–c。公元100年）为了与汉人结成婚姻同盟，在公元90年遭到拒绝，他派遣军队前往瓦罕（

,title,question,answer,context
18746,中世纪,为什么国王会在这期间提高税率？,为战争买单,中世纪晚期，强大的、以皇室为基础的民族国家在整个欧洲兴起，特别是在英格兰、法国和伊比利亚半岛...


In [40]:
inference_100('继西班牙人之后，谁是最早定居北卡罗来纳州的欧洲人？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
继西班牙人之后，谁是最早定居北卡罗来纳州的欧洲人？

检索结果：
由布宜诺斯艾利斯市西班牙总督布鲁诺·莫里西奥·德扎巴拉组织的西班牙探险队从布宜诺斯艾利斯出发。1724年1月22日，西班牙人强迫葡萄牙人放弃这一地点，开始在布宜诺斯艾利斯居住，最初有6个家庭从布宜诺斯艾利斯搬来，随后不久，从加那利群岛来的家庭被当地人称为“关卡人”、“关卡人”或“加那利人”。还有一位重要的早期意大利居民名叫豪尔赫·伯格斯。
土著美国人和非裔美国人之间的种族间关系是美国历史上被忽视的一部分。美洲最早的非洲和土著美洲关系记录发生在1502年4月，当时第一批被绑架的非洲人被带到伊斯帕尼奥拉当奴隶。一些人逃走了，在圣多明各内陆的某个地方，第一批黑人印第安人诞生了。此外，非洲奴隶逃离欧洲殖民者并被美洲土著吸收的例子早在1526年就出现了。同年6月，卢卡斯·瓦斯奎兹·德·艾伦在现在南卡罗来纳州东部的皮迪河口附近建立了一个西班牙殖民地。西班牙的定居点被命名为圣米格尔·德瓜尔达佩。定居点中有100名被奴役的非洲人。1526年，第一批非洲奴隶逃离殖民地，到当地的美洲土著人那里避难。
英国裔美国人和苏格兰裔美国人爱尔兰裔美国人开始从佐治亚州和南卡罗来纳州的边远地区迁入北佛罗里达州。尽管技术上西班牙当局不允许，但西班牙人始终无法有效地控制边境地区，来自美国的边远地区定居者将继续不受限制地迁移到佛罗里达州。这些移民与已经存在的英国殖民者混合，这些殖民者是英国时期以来一直留在佛罗里达州的移民，他们是佛罗里达饼干的祖先。
公元200年以前，居民们正在建造土堆，用于仪式和宗教目的。后来的人，包括公元1000年在山前建立的古密西西比文化的人，继续在这些土丘上建造或增加。在欧洲接触之前的500-700年里，密西西比文化建立了大型、复杂的城市，并保持着广泛的区域贸易网络。北卡罗莱纳州地区历史上记载的部落包括沿海地区讲卡罗莱纳阿尔冈琴语的部落，如Chowanoke、Roanoke、Pamlico、Machapunga、Coree、Cape Fear印第安人等，他们是第一个接触英语的部落；内陆地区讲易洛魁语的Meherrin、Cherokee和Tuscarora部落；以及东南部的苏族人，如切罗族、瓦肖族、萨皮奥族、瓦卡马族和卡塔瓦族。[需要引用]
有证据表明，土著人早在上一个冰河时代就

,title,question,answer,context
51839,北卡罗来纳州,继西班牙人之后，谁是最早定居北卡罗来纳州的欧洲人？,英国殖民者,16世纪西班牙人之后，北卡罗来纳州的第一批欧洲永久定居者是从弗吉尼亚南迁的英国殖民者。后者发...


In [64]:
inference_100('继西班牙人之后，谁是最早定居北卡罗来纳州的欧洲人？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
继西班牙人之后，谁是最早定居北卡罗来纳州的欧洲人？

检索结果：
由布宜诺斯艾利斯市西班牙总督布鲁诺·莫里西奥·德扎巴拉组织的西班牙探险队从布宜诺斯艾利斯出发。1724年1月22日，西班牙人强迫葡萄牙人放弃这一地点，开始在布宜诺斯艾利斯居住，最初有6个家庭从布宜诺斯艾利斯搬来，随后不久，从加那利群岛来的家庭被当地人称为“关卡人”、“关卡人”或“加那利人”。还有一位重要的早期意大利居民名叫豪尔赫·伯格斯。
土著美国人和非裔美国人之间的种族间关系是美国历史上被忽视的一部分。美洲最早的非洲和土著美洲关系记录发生在1502年4月，当时第一批被绑架的非洲人被带到伊斯帕尼奥拉当奴隶。一些人逃走了，在圣多明各内陆的某个地方，第一批黑人印第安人诞生了。此外，非洲奴隶逃离欧洲殖民者并被美洲土著吸收的例子早在1526年就出现了。同年6月，卢卡斯·瓦斯奎兹·德·艾伦在现在南卡罗来纳州东部的皮迪河口附近建立了一个西班牙殖民地。西班牙的定居点被命名为圣米格尔·德瓜尔达佩。定居点中有100名被奴役的非洲人。1526年，第一批非洲奴隶逃离殖民地，到当地的美洲土著人那里避难。
从西伯利亚穿过白令陆桥进入北美的移民可能有狗陪伴，一位作家认为，雪橇狗的使用可能是大约12000年前进入北美的海浪成功的关键，尽管北美最早的狗样犬科动物考古证据可以追溯到大约9400年前。：104条狗是北美阿萨巴斯卡人生活的重要组成部分，也是他们唯一的家养动物。1400年前，在阿帕奇和纳瓦霍部落的迁徙中，狗也承担了大部分的负荷。在这些文化中，狗作为驮畜的使用在马被引入北美后经常持续。
英国裔美国人和苏格兰裔美国人爱尔兰裔美国人开始从佐治亚州和南卡罗来纳州的边远地区迁入北佛罗里达州。尽管技术上西班牙当局不允许，但西班牙人始终无法有效地控制边境地区，来自美国的边远地区定居者将继续不受限制地迁移到佛罗里达州。这些移民与已经存在的英国殖民者混合，这些殖民者是英国时期以来一直留在佛罗里达州的移民，他们是佛罗里达饼干的祖先。
土著遗传研究表明，美洲的第一批居民有一个共同的祖先群体，一个在孤立中发展起来的，推测是白令病。这些人在白令岛的隔离可能持续了10-20000年。大约在16500年前，冰川开始融化，人们可以向南和向东迁移到加拿大和更远的地方。据信，这些人沿着劳伦蒂德和科迪勒兰冰原之间的无冰走廊

,title,question,answer,context
51839,北卡罗来纳州,继西班牙人之后，谁是最早定居北卡罗来纳州的欧洲人？,英国殖民者,16世纪西班牙人之后，北卡罗来纳州的第一批欧洲永久定居者是从弗吉尼亚南迁的英国殖民者。后者发...


In [41]:
inference_100('芽的发育是如何决定的？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
芽的发育是如何决定的？

检索结果：
种皮的性状与果实的性状有一定的关系。它们保护胚胎并帮助传播；它们也可以直接促进发芽。一般来说，在有不裂果实的植物中，果实为胚胎提供保护并确保传播。在这种情况下，种皮只稍微发育。如果果实开裂，种子外露，一般来说，种皮发育良好，必须履行果实的其他功能。
根系和地上部系统是相互依存的，通常不光合的根系依赖于地上部系统作为食物，而通常光合的地上部系统依赖于来自根系的水和矿物质。每个系统中的细胞都能产生另一个系统的细胞，并产生不定的芽或根。匍匐茎和块茎是能生根的芽的例子。像柳树这样的根靠近地表伸展，可以产生嫩枝，最终产生新的植物。如果其中一个系统丢失，另一个系统通常会重新生长。事实上，从一片叶子上长出一整株植物是可能的，就像圣保利亚的情况一样，甚至是一个细胞——它可以脱分化为一个愈伤组织（大量未分化的细胞），可以长出一个新的植物。在维管植物中，木质部和韧皮部是在茎和根之间运输资源的传导组织。根通常被用来储存食物，如糖或淀粉，如甜菜和胡萝卜。
孢子化石表明高等植物（胚胎植物）在陆地上至少生活了4.75亿年。早期的陆地植物通过有鞭毛、游动的精子进行有性繁殖，就像它们进化的绿藻一样。对陆地化的一种适应是直立的梅osporangia的发展，以便通过孢子传播到新的栖息地。这种特征在它们最近的藻类近亲轮藻绿藻的后代中是缺乏的。后来的陆生适应发生在维管孢子体的组织中，保留了脆弱的无血管性阶段，即配子体。这发生在孢子囊内的孢子萌发而不是孢子释放，如在非种子植物中。卷柏属（spike moss）的早熟孢子萌发就是一个很好的例子。被子植物祖先的结果是把它们包在一个案例中，即种子。最初的种子植物，如银杏和针叶树（如松树和冷杉），不开花。银杏和苏铁的花粉粒（雄性）产生一对有鞭毛的、可移动的精子细胞，它们沿着发育中的花粉管“游”到雌性和她的卵细胞。
真核生物中的表观遗传变化起着调节细胞分化过程的作用。在形态发生过程中，全能干细胞成为胚胎的各种多能干细胞系，进而成为全分化细胞。一个受精卵细胞，合子，产生许多不同的植物细胞类型，包括薄壁组织、木质部导管分子、韧皮部筛管、表皮保卫细胞等。因为它继续分裂。这一过程是由于某些基因的表观遗传激活和其他基因的抑制。
与动物不同的是，许多植物细胞，特别是那些薄壁组织的细胞，没有最终分化，仍然

,title,question,answer,context
41615,植物,芽的发育是如何决定的？,胞质分裂,细胞分裂素是一类植物激素，因其控制细胞分裂或胞质分裂而得名。玉米中发现了天然的细胞分裂素-玉...


In [43]:
inference_100('2006年iPod在各种电脑产品中的排名是多少？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
2006年iPod在各种电脑产品中的排名是多少？

检索结果：
ipod获得了多项奖项，从卓越的工程设计[未被引用]到最具创新性的音频产品，再到2006年第四届最佳计算机产品。ipod通常会得到好评：外观评分、设计简洁、使用方便。PC世界说，iPod系列“改变了便携式音频播放器的格局”。一些行业正在修改他们的产品，以便更好地使用iPod系列和AAC音频格式。例如CD拷贝保护计划，以及手机，如索尼爱立信（Sony Ericsson）和诺基亚（Nokia）的手机，它们播放AAC文件而不是WMA。
2007年4月9日，苹果公司宣布其第一亿部iPod已经售出，成为有史以来销量最大的数字音乐播放器。2007年4月，苹果公布第二季度营收52亿美元，其中32%来自iPod销售。苹果和几位行业分析师认为，iPod用户可能会购买其他苹果产品，如Mac电脑。
许多配件都是为iPod生产的。大量的产品是由第三方公司生产的，尽管许多产品，如iPod高保真，都是由苹果公司生产的。一些附件添加了其他音乐播放器所具有的额外功能，如录音机、调频收音机调谐器、有线遥控器和用于电视连接的音频/视频电缆。其他配件提供独特的功能，如Nike+iPod计步器和iPod摄像头接口。其他值得注意的配件包括外置扬声器、无线遥控器、保护套、屏幕贴膜和无线耳机。首批配件制造商包括格里芬科技、贝尔金、JBL、Bose、Monster Cable和SendStation。
自2004年10月以来，iPod系列一直主导着美国数字音乐播放器的销售，90%以上的市场是基于硬盘驱动器的播放器，70%以上的市场是所有类型的播放器。在2004年1月至2005年1月期间，高销售率导致其美国市场份额从31%增加到65%，2005年7月，该市场份额为74%。根据彭博在线（Bloomberg Online）的数据，2007年1月，iPod的市场份额达到了72.7%。
和其他数字音乐播放器一样，ipod可以作为外部数据存储设备。存储容量因型号而异，从iPodShuffle的2GB到iPodtouch的128GB（以前iPodClassic的160GB，现在已经停产）。

回答：
['第 四']


,title,question,answer,context
987,IPod,2006年iPod在各种电脑产品中的排名是多少？,第四,ipod获得了多项奖项，从卓越的工程设计[未被引用]到最具创新性的音频产品，再到2006年第...


In [44]:
inference_100('什么宗教导致了许多亚洲国家的建筑变化？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
什么宗教导致了许多亚洲国家的建筑变化？

检索结果：
亚洲不同地区的建筑沿着与欧洲不同的路线发展；佛教、印度教和锡克教的建筑各有不同的特点。尤其是佛教建筑，表现出了巨大的地域多样性。印度教寺庙建筑，发展于公元前3世纪左右，是由沙斯特拉斯的概念所支配的，涉及表达宏观和微观。在许多亚洲国家，泛神论的宗教导致了建筑形式，是专门为增强自然景观而设计的。
伊斯兰建筑始于公元前7世纪，融合了古代中东和拜占庭的建筑形式，同时也发展出适应社会宗教和社会需要的特色。中东、北非、西班牙和印度次大陆都有这样的例子。尖拱的广泛应用影响了中世纪的欧洲建筑。
东南亚的宗教和民族多种多样，没有一个国家是同一的。在世界上人口最多的穆斯林国家印尼，印度教在巴厘岛等岛屿上占主导地位。基督教在菲律宾、新几内亚和帝汶的其他地区也占主导地位。新加坡、马来西亚等地的东南亚地区也有印度教人口。毗湿奴峰（vahanam）的凤凰城（Garuda，梵语：Garuḍa）是泰国和印度尼西亚的国家象征；在菲律宾，在巴拉望发现了Garuda的黄金图像；在棉兰老岛也发现了其他印度神和女神的黄金图像。巴厘的印度教与其他地方的印度教有所不同，因为万物有灵论和当地文化被融入其中。基督徒也可以在整个东南亚找到；他们在东帝汶和菲律宾，亚洲最大的基督教国家占多数。此外，马来西亚东部砂拉越、菲律宾高原和印尼东部巴布亚等偏远地区也有较古老的部落宗教习俗。在缅甸，萨卡（因陀罗）被尊为纳特。在越南，大乘佛教是实践，这是受本土万物有灵论的影响，但强烈强调祖先崇拜。
佛教逐渐向邻近地区传播，意味着它与新的民族群体接触。在这一时期，佛教受到各种各样的影响，从波斯和希腊文明，到印度非佛教宗教自身受佛教影响的变化趋势。这种融合发展的显著例子可以从印度-希腊王国中讲希腊语的佛教君主的出现，以及甘地的希腊佛教艺术的发展中看出。希腊国王梅南德甚至在佛经中被永生。
有人指出，高棉和印度尼西亚的古典艺术关注的是描绘神的生命，但在东南亚人的心目中，神的生命是人民自己的生命，快乐，朴实，但神圣。泰人后来进入东南亚，带来了一些中国的艺术传统，但他们很快就抛弃了这些传统，取而代之的是高棉和孟族的传统，他们早期接触中国艺术的唯一迹象是寺庙的风格，特别是尖顶和漆器。

回答：
['伊 斯 兰 教']


,title,question,answer,context
6612,建筑,什么宗教导致了许多亚洲国家的建筑变化？,泛神论,亚洲不同地区的建筑沿着与欧洲不同的路线发展；佛教、印度教和锡克教的建筑各有不同的特点。尤其是...


In [65]:
inference_100('什么宗教导致了许多亚洲国家的建筑变化？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
什么宗教导致了许多亚洲国家的建筑变化？

检索结果：
亚洲不同地区的建筑沿着与欧洲不同的路线发展；佛教、印度教和锡克教的建筑各有不同的特点。尤其是佛教建筑，表现出了巨大的地域多样性。印度教寺庙建筑，发展于公元前3世纪左右，是由沙斯特拉斯的概念所支配的，涉及表达宏观和微观。在许多亚洲国家，泛神论的宗教导致了建筑形式，是专门为增强自然景观而设计的。
伊斯兰建筑始于公元前7世纪，融合了古代中东和拜占庭的建筑形式，同时也发展出适应社会宗教和社会需要的特色。中东、北非、西班牙和印度次大陆都有这样的例子。尖拱的广泛应用影响了中世纪的欧洲建筑。
主要的建筑工程是修道院和大教堂的建筑。从公元前900年开始，神职人员和商人的运动将建筑知识带到了整个欧洲，形成了泛欧洲风格的罗马式和哥特式。
亚洲早期的建筑著作包括公元前7-5世纪的中国高工集、古印度的夏尔巴沙斯特拉和斯里兰卡的文殊瓦苏-维迪亚-萨斯特拉。
中世纪早期索马里历史上伊斯兰教的和平引入，带来了阿拉伯和波斯伊斯兰建筑的影响。这刺激了建筑业从干石头和其他相关材料向珊瑚石、日晒砖的转变，以及石灰石在索马里建筑中的广泛使用。许多新的建筑设计，如清真寺，都是在旧建筑的废墟上建造的。在接下来的几个世纪里，这种做法将一次又一次地继续下去。

回答：
['泛 神 论 的 宗 教']


,title,question,answer,context
6612,建筑,什么宗教导致了许多亚洲国家的建筑变化？,泛神论,亚洲不同地区的建筑沿着与欧洲不同的路线发展；佛教、印度教和锡克教的建筑各有不同的特点。尤其是...


In [45]:
inference_100('英国足球俱乐部在哪一年被禁止在欧洲比赛？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
英国足球俱乐部在哪一年被禁止在欧洲比赛？

检索结果：
由于担心俱乐部越来越多地将年轻的英格兰球员让渡给外国球员，内政部在1999年收紧了向欧盟以外国家球员发放工作许可证的规定。申请该许可证的非欧盟球员必须在过去两年内参加过至少75%的可供选择的竞争性“A”球队比赛，并且他的国家在过去两年的国际足联世界排名中平均至少排名第70位。如果球员不符合这些标准，希望签下他的俱乐部可以上诉。
尽管欧洲在20世纪70年代和80年代初取得了巨大的成功，但80年代末标志着英国足球的低谷。体育场摇摇欲坠，球迷忍受着糟糕的设施，流氓行为盛行，英格兰俱乐部在1985年海塞尔体育场灾难后5年内被禁止参加欧洲比赛。足球甲级联赛自1888年以来一直是英国足球的顶级联赛，在出勤率和收入方面远远落后于意大利甲级联赛和西班牙西甲联赛，几名英国顶级球员已经移居国外。
英格兰在欧洲足球锦标赛上是一个相当成功的国家，1968年获得第三名，1996年进入半决赛。英格兰主办了96年欧洲杯，并参加了8次欧冠决赛，并列第九。该队最近在2004年和2012年两次进入四分之一决赛。该队在比赛中最差的成绩是在1980年、1988年、1992年和2000年的第一轮淘汰赛。这支队伍在1960年没有进入，他们在1964年、1972年、1976年、1984年和2008年都没有获得参赛资格。
然而，到了90年代初，下降的趋势开始逆转；英格兰在1990年世界杯上取得了成功，进入了半决赛。欧洲足球管理机构欧足联（UEFA）于1990年取消了英国俱乐部在欧洲比赛中的五年禁赛（导致曼联在1991年取消了欧足联杯冠军杯）和《泰勒体育场安全标准报告》，在希尔斯伯勒灾难之后，该计划提出了昂贵的升级，以建造所有座位的体育场馆，并于当年1月公布。
英国在1974年、1978年和1994年没有获得世界杯的参赛资格。球队在比赛中最早的退出是在1950, 1958的第一轮淘汰赛，最近是在2014国际足联世界杯上，他们在首场两场比赛中被击败，而在D组中意大利队和乌拉圭队在1950场比赛中，四支球队在第一轮比赛后一直保持下去，1958支八支球队仍然留在比赛中。2014年仍有16支球队。2010年，英格兰队在16轮世界杯上遭遇了最惨烈的一场失利（4比1负于德国），此前英格兰队与美国和阿尔及利亚队战平，小组赛1比0击败斯

,title,question,answer,context
76846,英超联赛,英国足球俱乐部在哪一年被禁止在欧洲比赛？,1985,尽管欧洲在20世纪70年代和80年代初取得了巨大的成功，但80年代末标志着英国足球的低谷。体...


In [46]:
inference_100('电阻可以连接在哪里？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
电阻可以连接在哪里？

检索结果：
电容器可能会在电路断电后很长时间内保持电荷；这种电荷可能会导致危险的甚至潜在的致命冲击或损坏连接的设备。例如，即使是一个看似无害的设备，例如一个由1.5伏a a电池供电的一次性照相机闪光灯，也有一个电容器，电容器可能包含超过15焦耳的能量，充电到300伏以上。这很容易引起震动。电子设备的维修程序通常包括大或高压电容器放电的说明，例如使用Brinkley棒。电容器也可能有内置的放电电阻，以便在断电后几秒钟内将储存的能量耗散到安全水平。高压电容器与短路端子一起储存，以防止因介质吸收而产生的潜在危险电压或电容器可能从静电荷或经过的天气事件中获取的瞬态电压。
两种类型的充电端口存在：充电的下行端口（CDP），支持数据传输，以及专用的充电端口（DCP），没有数据支持。便携式设备可以识别USB端口的类型；在专用充电端口上，D+和D-引脚被短路，电阻不超过200欧姆，而充电下游端口提供额外的检测逻辑，因此它们的存在可以由连接的设备确定。（本参考页。2，第1.4.5节和表5-3“电阻”-pg。29).
在配电系统中，电容器用于功率因数校正。这种电容器通常是作为三相负载连接的三个电容器。通常，这些电容器的值不是以法拉为单位，而是以伏安无功（var）为单位的无功功率。其目的是抵消电动机和输电线路等设备产生的感应负载，使负载看起来主要是电阻性的。单个电机或灯具负载可能具有用于功率因数校正的电容器，或者更大的电容器组（通常带有自动开关装置）可能安装在建筑物内的负荷中心或大型公用变电站中。
空气介质电容器的击穿场强约为2～5mv/m；云母的击穿场强约为100～300mv/m；石油的击穿场强约为15～25mv/m；使用其他材料作介质时，击穿场强可小得多。电介质用于非常薄的层，因此电容器的绝对击穿电压是有限的。一般电子设备用电容器的典型额定值从几伏到1千伏不等。随着电压的增加，电介质必须更厚，使得高压电容器的每一个电容比那些额定电压较低的电容器更大。击穿电压受到电容器导电部件几何形状等因素的严重影响；尖锐的边缘或点会增加该点的电场强度，并可能导致局部击穿。一旦这种情况开始发生，击穿会迅速通过电介质，直到它到达相反的极板，留下碳并导致短路（或电阻相对较低）。当电容器中的短路从周围电路中吸取电流并耗散能量时，结果可能是爆炸性

,title,question,answer,context
46216,电动机,电阻可以连接在哪里？,转子电路,转子绕组是由多匝绝缘线组成，与电机轴上的滑环相连。转子电路中可连接外部电阻器或其他控制装置。...


In [47]:
inference_100('什么物种通常被猎杀？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
什么物种通常被猎杀？

检索结果：
种群规模小是一个几乎普遍存在的特征，对于顶级捕食者来说，人类和狗是迄今为止最明显的例外。如果有大量的猎物而没有竞争或生态位重叠，那么数量少对顶级捕食者来说就不是问题了，这种情况在野外很少会遇到。竞争排斥原则指出，如果两个物种的生态位重叠，则很有可能发生竞争，因为两个物种都在直接竞争同一资源。这一因素本身可能导致一个或两个物种灭绝，但同时又增加了猎物数量的因素。
社会捕食提供了一种可能性，即捕食者杀死的生物比该物种成员能够单独战胜的生物大。狮子、鬣狗、狼、洞狗、非洲野狗和食人鱼可以杀死大型食草动物，而同一物种的单一动物通常不会杀死它们。社会捕食使一些动物能够组织捕猎容易逃离单一捕食者的生物；因此黑猩猩可以捕食疣猴，哈里斯的鹰可以为一只注定要死去的兔子切断一切可能的逃跑。在一些需要不同种类捕食者合作的狩猎中，角色的极端特殊化是显而易见的：人类借助于猎鹰或狗，或是用鸬鹚捕鱼。社会捕食通常是非常复杂的行为，并不是所有的社会生物（例如家猫）都会这样做。即使没有复杂的智力，只有本能，一些蚂蚁物种也能摧毁更大的生物。
捕食者之间有很大程度的专门化。许多捕食者只捕猎一种猎物。其他人则更机会主义，几乎会杀死和吃掉任何东西（例如：人类、豹子、狗和鳄鱼）。专家们通常特别适合捕捉他们喜欢的猎物。反过来，猎物通常也同样适合躲避捕食者。这被称为进化军备竞赛，倾向于保持两个物种的种群平衡。有些捕食者专门捕食某些种类的猎物，而不仅仅是单一的物种。当首选目标极为稀少时，一些人会转向其他猎物（成功程度不同），如果可能，他们也可能求助于食腐或草食性食物。[需要引用]
很明显，捕食者倾向于降低其猎物的生存和繁殖力，但在更高的组织层次上，捕食者种群和被捕食物种也相互作用。很明显，捕食者依靠猎物生存，这反映在捕食者种群受到猎物种群变化的影响。然而，捕食者影响猎物数量的现象并不明显。如果猎物数量接近其承载能力，吃一个被捕食的有机体只会为另一个有机体腾出空间。
至少从20世纪初以来，物种入侵的数量一直在增加。越来越多的物种被人类（有意或无意）移动。在某些情况下，入侵者正在对它们的新栖息地造成剧烈的变化和破坏（例如。：大湖区的斑马贻贝和翡翠灰蛀虫以及北美大西洋沿岸的狮子鱼）。一些证据表明，入侵物种在它们的新栖息地具有竞争力，因为它们受到的病

,title,question,answer,context
89262,狩猎,什么物种通常被猎杀？,哺乳动物和鸟类,捕猎是指捕杀或诱捕任何动物，或追求或追踪任何动物的行为。猎杀野生动物或野生动物通常是人类为了...


In [66]:
inference_100('什么物种通常被猎杀？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
什么物种通常被猎杀？

检索结果：
捕猎是指捕杀或诱捕任何动物，或追求或追踪任何动物的行为。猎杀野生动物或野生动物通常是人类为了食物、娱乐、清除对人类或家畜有危险的捕食者或为了交易而进行的。在20世纪10年代，合法狩猎不同于偷猎，偷猎是指非法捕杀、诱捕或捕获被猎杀的物种。被猎杀的物种被称为猎物或猎物，通常是哺乳动物和鸟类。
捕食是一种生物相互作用，捕食者（捕猎的异养动物）以猎物（被攻击的有机体）为食。捕食者在捕食猎物之前可能会或不会杀死它们，但捕食行为几乎总是导致猎物死亡。另一种主要的消费是有害的，即消耗死的有机物。有时很难区分这两种取食行为，例如，寄生物种捕食宿主有机体，然后在其上产卵，让其后代以其腐烂的尸体为食。相互施加的选择性压力导致了猎物和捕食者之间的进化军备竞赛，导致了各种反转录因子的适应。
狼和大猫也被认为会攻击狗。众所周知，美洲豹特别喜欢狗，有记录表明，无论狗的大小或凶猛程度如何，美洲豹都会杀死并吃掉它们。满洲里、印度支那、印度尼西亚和马来西亚的老虎被认为能像豹一样杀死狗。条纹鬣狗是土库曼斯坦、印度和高加索地区乡村狗的主要捕食者。众所周知，鳄鱼和蟒蛇等爬行动物会杀死和吃掉狗。
社会捕食提供了一种可能性，即捕食者杀死的生物比该物种成员能够单独战胜的生物大。狮子、鬣狗、狼、洞狗、非洲野狗和食人鱼可以杀死大型食草动物，而同一物种的单一动物通常不会杀死它们。社会捕食使一些动物能够组织捕猎容易逃离单一捕食者的生物；因此黑猩猩可以捕食疣猴，哈里斯的鹰可以为一只注定要死去的兔子切断一切可能的逃跑。在一些需要不同种类捕食者合作的狩猎中，角色的极端特殊化是显而易见的：人类借助于猎鹰或狗，或是用鸬鹚捕鱼。社会捕食通常是非常复杂的行为，并不是所有的社会生物（例如家猫）都会这样做。即使没有复杂的智力，只有本能，一些蚂蚁物种也能摧毁更大的生物。
食肉动物经常用它们通常的攻击猎物的方法来对自己的食肉动物造成或威胁造成严重伤害。电鳗利用同样的电流杀死猎物，并保护自己免受通常捕食与电鳗大小相似的鱼类的动物（水蟒、凯门鳄、白鹭、美洲虎、山狮、巨型水獭、人类、狗和猫）的侵害；因此，电鳗在捕食者丰富的环境中仍然是一种顶级捕食者。家猫是一种体型小到可以捕食其他动物的食肉动物，它用它那可怕的牙齿和爪子作为武器来对付那些可能把猫和更容易被捕食的动物混淆的动

,title,question,answer,context
89262,狩猎,什么物种通常被猎杀？,哺乳动物和鸟类,捕猎是指捕杀或诱捕任何动物，或追求或追踪任何动物的行为。猎杀野生动物或野生动物通常是人类为了...


In [49]:
inference_100('2000年有多少家庭有孩子？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
2000年有多少家庭有孩子？

检索结果：
2000年，在45693户家庭中，有23.0%的家庭有18岁以下的子女，37.8%的家庭是已婚夫妇，7.5%的家庭是女性，没有丈夫在场，52.5%的家庭是非家庭成员。35.5%的家庭由个人组成，6.6%的家庭有65岁以上的独居者。平均家庭规模为2.22户，平均家庭规模为2.90户。年龄分布：18岁以下16.8%，18～24岁26.8%，25～44岁31.2%，45～64岁17.3%，65岁及以上7.9%。中位年龄为28岁。每100名女性中有97.7名男性；每100名18岁及以上女性中有96.4名男性。
截至2010年[更新]，共有366,273户家庭，其中11.8％处于空置状态。23.9％的家庭中有18岁以下的儿童与父母同住，43.8％的家庭是已婚夫妇，15.2％的女性家庭中没有丈夫，并且36.4％的家庭为非家庭。有单独居住的某人的29.7％所有家庭由个体做成和7.9％谁65岁或更旧。平均家庭大小是2.55，并且平均家庭大小是3.21。在城市中，未满18岁的人口比例为23.9％，18岁至24岁的人口比例为10.5％，25岁至44岁的人口比例为28.5％，45岁至64岁的人口比例为26.2％，65岁以上的人口比例为10.9％ 中位年龄是35.5岁。每100名女性中有94.1名男性。每100名18岁及以上的女性中，男性为91.3。
有158,349户家庭，其中68,511（43.3％）个年龄在18岁以下的儿童居住在其中； 69,284（43.8％）岁的异性伴侣在一起生活，30,547户（19.3％）的女性家庭没有丈夫有12,843个（8.1％）未婚异性伴侣关系，以及1,388个（0.9％）同性已婚夫妇或伴侣关系。有单独居住的某人的35,064户家庭（22.1％）由个体组成，并且65岁以上的人为12,344（7.8％）。平均家庭大小是3.07。有111,529户家庭（占所有家庭的70.4％）；
有230233户家庭，其中有18岁以下子女的家庭占29.4%，有配偶的家庭占43.4%，有无丈夫的女户主的家庭占13.9%，无丈夫的家庭占37.4%。一人家庭占所有家庭的30.5%，8.7%的家庭有65岁以上的独居者。平均家庭规模为2.47，平均家庭规模为3.11。
全市人口分布，19岁及以下人口占

,title,question,answer,context
46859,北卡罗来纳州罗利,2000年有多少家庭有孩子？,26.5%,2000年全市共有家庭112608户，其中18岁以下儿童占26.5%，同居夫妻占39.5%，...


In [67]:
inference_100('2000年有多少家庭有孩子？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
2000年有多少家庭有孩子？

检索结果：
有230233户家庭，其中有18岁以下子女的家庭占29.4%，有配偶的家庭占43.4%，有无丈夫的女户主的家庭占13.9%，无丈夫的家庭占37.4%。一人家庭占所有家庭的30.5%，8.7%的家庭有65岁以上的独居者。平均家庭规模为2.47，平均家庭规模为3.11。
2000年，在45693户家庭中，有23.0%的家庭有18岁以下的子女，37.8%的家庭是已婚夫妇，7.5%的家庭是女性，没有丈夫在场，52.5%的家庭是非家庭成员。35.5%的家庭由个人组成，6.6%的家庭有65岁以上的独居者。平均家庭规模为2.22户，平均家庭规模为2.90户。年龄分布：18岁以下16.8%，18～24岁26.8%，25～44岁31.2%，45～64岁17.3%，65岁及以上7.9%。中位年龄为28岁。每100名女性中有97.7名男性；每100名18岁及以上女性中有96.4名男性。
2000年全市共有家庭112608户，其中18岁以下儿童占26.5%，同居夫妻占39.5%，无夫女户主占11.4%，非婚家庭占45.5%。2.2%的家庭有未婚伴侣。此外，33.1%的家庭由独居者组成，其中6.2%是65岁以上的人。罗利市平均家庭规模为2.30人，平均家庭规模为2.97人。
家庭15504户，其中有18岁以下子女的家庭占27.3%，夫妻共同生活的家庭占25.9%，有无丈夫的女户主的家庭占22.2%，无丈夫的家庭占44.8%。37.5%的家庭由个人组成，14.3%的家庭有65岁以上的独居者。平均家庭规模为2.50，平均家庭规模为3.34。
有15848户家庭，其中有18岁以下子女的家庭占27.7%，有夫妻共同生活的家庭占24.8%，有无丈夫的女户主的家庭占23.2%，无丈夫的家庭占45.1%。37.2%的家庭由个人组成，15.4%的家庭有65岁以上的独居者。平均家庭规模为2.46，平均家庭规模为3.26。

回答：
['三 分 之 一']


,title,question,answer,context
46859,北卡罗来纳州罗利,2000年有多少家庭有孩子？,26.5%,2000年全市共有家庭112608户，其中18岁以下儿童占26.5%，同居夫妻占39.5%，...


In [50]:
inference_100('戴尔是哪一年降价以保持市场份额的？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
戴尔是哪一年降价以保持市场份额的？

检索结果：
戴尔的广告出现在电视、互联网、杂志、目录和报纸等多种媒体上。戴尔公司的一些营销策略包括在一年中的任何时候降低价格、免费赠送产品（如戴尔打印机）和免费送货，以鼓励更多的销售和避开竞争对手。2006年，戴尔为了保持其19.2%的市场份额而降价。这也使利润率下降了一半以上，从8.7%降至4.3%。为了维持低价格，戴尔继续接受通过互联网和电话网络购买的大部分产品，并将其客户服务部门迁往印度和萨尔瓦多。
2007年8月17日，戴尔公司宣布，在对其会计实务进行内部调查后，将重述并减少2003年至2007年第一季度的收入，总额在5000万至1.5亿美元之间，即每股2美分至7美分。调查始于2006年11月，起因是美国证券交易委员会（U.S.Securities and Exchange Commission）对戴尔公司（Dell Inc.）提交的一些文件和信息表示担忧。据称，戴尔并未披露因同意不从竞争对手AMD购买处理器而从英特尔收到的巨额独家付款。2010年，戴尔最终支付了1亿美元，以解决美国证交会的欺诈指控。迈克尔戴尔和其他高管也支付了罚款，并遭受了其他制裁，但没有承认或否认指控。
戴尔于2004年成为信息技术行业第一家确立产品回收目标的公司，并于2006年完成了全球消费者回收计划的实施。2007年2月6日，国家回收联盟授予戴尔公司“回收工程”奖，表彰其为促进生产者责任所做的努力。2007年7月19日，戴尔宣布，在努力实现到2009年收回2.75亿磅计算机设备的多年目标方面，已经超额完成了目标。该公司报告说，2006年从客户那里回收了7800万英镑（近40000吨）的IT设备，比2005年增长了93%；戴尔7年前售出的设备中，有12.4%的设备被回收。
2009年7月，戴尔因两次拒绝履行其台湾网站上异乎寻常的低价订单而招致台湾消费者保护委员会（taiwan Consumer Protection Commission）的愤怒，戴尔为此道歉。在第一个例子中，戴尔以15美元的价格提供了一个19英寸的液晶面板。在第二个例子中，戴尔提供的Latitude4300笔记本电脑售价为18558新台币（580美元），比通常的60900新台币（1900美元）低70%。关于E4300，该公司没有兑现遭受重大损

,title,question,answer,context
30535,戴尔,戴尔是哪一年降价以保持市场份额的？,2006,戴尔的广告出现在电视、互联网、杂志、目录和报纸等多种媒体上。戴尔公司的一些营销策略包括在一年...


In [51]:
inference_100('谁写了《塞尔伯恩自然史》？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
谁写了《塞尔伯恩自然史》？

检索结果：
1844年11月，苏格兰记者罗伯特钱伯斯（Robert Chambers）撰写的一本匿名出版的科普书《自然创造史的遗迹》，扩大了公众对物种嬗变概念的兴趣。遗迹利用化石记录和胚胎学的证据来支持生物随着时间的推移从简单进化到更复杂的说法。但它提出了一个线性级数，而不是达尔文正在进行的工作背后的分支共同下降理论，它忽略了适应。达尔文在发表后不久就阅读了这本书，并蔑视它的业余地质学和动物学，但在包括亚当·塞奇威克在内的著名科学家抨击它的道德和科学错误之后，他仔细回顾了自己的论点。遗迹对公众舆论产生了重大影响，激烈的辩论有助于通过将进化论的猜测纳入主流，为接受更为科学复杂的起源铺平道路。尽管很少有博物学家愿意考虑变形，赫伯特·斯宾塞在19世纪50年代成为喇嘛教和进步发展的积极支持者。
伊本·图法尔的作品《斐罗索夫斯·奥托迪达克图》的拉丁文译本最早出现于1671年，由年轻的爱德华·波科克（Edward Pocoke）编写，随后于1708年由西蒙·奥克利（Simon Ockley）翻译成英文，以及德文和荷文译本。这些翻译可能会激励丹尼尔·笛福创作《鲁滨逊漂流记》，被认为是第一部英文小说。自我陶醉哲学延续了早期亚里士多德等哲学家的思想，激发了罗伯特·博伊尔以一个岛屿为背景创作自己的哲学小说，这位有抱负的自然主义者。
1816年拜伦、珀西·比舍·雪莱和他的妻子玛丽·雪莱访问了日内瓦，三人都受到了他们作品中的风景的启发。在这些访问中，雪莱写了诗《勃朗峰》，拜伦写了诗《齐伦的囚徒》和戏剧诗《曼弗雷德》，玛丽雪莱发现景色让人无法抗拒，于是在雷雨中，她在日内瓦湖畔的别墅里构思了小说《弗兰肯斯坦》。当柯勒律治前往查莫尼克斯时，他不顾雪莱在蒙滕弗斯附近的朗德雷斯酒店的留言簿上签上“雅典娜”的名字，大声说：“谁会，谁会成为这个奇迹谷的无神论者。”。到了19世纪中叶，科学家们开始成群结队地来到这里研究该地区的地质和生态。
杰弗里·乔叟（Geoffrey Chaucer）14世纪晚期的坎特伯雷故事中的朝圣者从伦敦出发前往坎特伯雷，特别是从南华克的塔巴德酒店出发。威廉·莎士比亚一生的很大一部分时间都在伦敦生活和工作；他同时代的本·琼森也在伦敦工作，他的一些作品，尤其是他的戏剧《炼金术士》是以伦敦为背景的。丹尼尔·笛福的《瘟

,title,question,answer,context
19509,鸟类迁徙,谁写了《塞尔伯恩自然史》？,吉尔伯特·怀特,亚里士多德却提出燕子和其他鸟类冬眠。这种信念一直持续到1878年，当时埃利奥特库斯列出了不少...


In [68]:
inference_100('谁写了《塞尔伯恩自然史》？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
谁写了《塞尔伯恩自然史》？

检索结果：
1844年11月，苏格兰记者罗伯特钱伯斯（Robert Chambers）撰写的一本匿名出版的科普书《自然创造史的遗迹》，扩大了公众对物种嬗变概念的兴趣。遗迹利用化石记录和胚胎学的证据来支持生物随着时间的推移从简单进化到更复杂的说法。但它提出了一个线性级数，而不是达尔文正在进行的工作背后的分支共同下降理论，它忽略了适应。达尔文在发表后不久就阅读了这本书，并蔑视它的业余地质学和动物学，但在包括亚当·塞奇威克在内的著名科学家抨击它的道德和科学错误之后，他仔细回顾了自己的论点。遗迹对公众舆论产生了重大影响，激烈的辩论有助于通过将进化论的猜测纳入主流，为接受更为科学复杂的起源铺平道路。尽管很少有博物学家愿意考虑变形，赫伯特·斯宾塞在19世纪50年代成为喇嘛教和进步发展的积极支持者。
作为一个学术领域，科学史始于威廉·惠威尔的《归纳科学史》（1837年首次出版）。乔治·萨顿的出版物《科学史导论》（1927年）和《Isis期刊》（1912年创办）对科学史作为一门独立学科展开了更为正式的研究。萨顿将20世纪早期的科学史视为伟人和伟大思想的历史。他与许多同时代的人分享了一种对历史的轻率信仰，作为进步进程中进步和延迟的记录。在这一时期，科学史并不是美国历史上公认的一个分支，大部分工作是由感兴趣的科学家和医生而不是专业历史学家完成的。1945年后，随着哈佛大学伯纳德·科恩（I.Bernard Cohen）的研究，科学史成为历史学科的一个分支。
1849年，第一本“现代”教科书马蒂亚斯·施莱登（Matthias Schleiden）的《科学植物学原理》（Grundzüge der Wissenschaftlichen Botanik）以英语出版，极大地促进了植物学的发展。施莱登是一位显微镜学家和早期的植物解剖学家，他与西奥多·施万和鲁道夫·维肖共同创立了细胞理论，是最早掌握罗伯特·布朗在1831年描述的细胞核意义的人之一。1855年，阿道夫·菲克（Adolf Fick）制定了菲克定律，从而能够计算生物系统中分子扩散的速率。
1859年11月24日出版的《物种起源》是Charles Darwin的一部科学文献，被认为是进化生物学的基础。达尔文的书介绍了一种科学理论，即种群在经过自然选择的过程中，经过几代人的进

,title,question,answer,context
19509,鸟类迁徙,谁写了《塞尔伯恩自然史》？,吉尔伯特·怀特,亚里士多德却提出燕子和其他鸟类冬眠。这种信念一直持续到1878年，当时埃利奥特库斯列出了不少...


In [52]:
inference_100('阿富汗南部的主要战斗部队是什么？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
阿富汗南部的主要战斗部队是什么？

检索结果：
塔利班在巴基斯坦西部重新集结，并于2002年底开始对联军发动叛乱式的进攻。在阿富汗南部和东部，不断壮大的塔利班和联军之间爆发了交火。联军采取了一系列军事攻势，并增加了在阿富汗的兵力。2010年2月，联军在阿富汗南部发动了摩什塔拉克行动和其他军事攻势，希望他们能一劳永逸地摧毁塔利班叛乱。与塔利班有联系的武装分子和联军也在进行和谈。2014年9月，阿富汗和美国签署了一项安全协议，允许美国和北约部队在阿富汗至少停留到2024年。美国和其他北约和非北约部队正计划撤军；塔利班声称已经击败了美国和北约，奥巴马政府视之为胜利。2014年12月，随着北约在阿富汗的行动开始，国际安全援助部队披上了自己的颜色，并获得了坚定的支持。美国将继续以“自由行动哨兵”的名义在阿富汗境内开展行动。
1979，苏联将第四十支军队部署到阿富汗，试图在阿富汗内战中镇压伊斯兰盟军反对伊斯兰主义的政权。这场冲突将土著贫困穆斯林（圣战者）与反宗教的超级大国对立起来，激发了全世界成千上万的穆斯林提供援助，有时甚至为了信仰而战。领导这一泛伊斯兰努力的是巴勒斯坦酋长阿卜杜拉·优素福·阿扎姆。虽然这些“阿富汗阿拉伯人”的军事效力微乎其微，但来自世界各地的大约16000至35000名穆斯林志愿者前来阿富汗作战。
在阿富汗，由于政治和部落军阀之间的恶性和破坏性内战，20世纪80年代圣战者战胜苏联并没有带来正义和繁荣，使阿富汗成为世界上最贫穷的国家之一。1992年，由共产党军队统治的阿富汗民主共和国解体，圣战者的民主伊斯兰分子建立了阿富汗伊斯兰国。1996年，一个更加保守和反民主的伊斯兰运动塔利班上台，击败了大部分军阀，占领了大约80%的阿富汗。
当美国在2002年底明确表示要入侵伊拉克时，对美国的支持就降温了。即便如此，许多无条件支持美国领导的军事行动的“自愿联盟”国家已向阿富汗，特别是邻国巴基斯坦派兵，后者已否认先前对塔利班的支持，并向冲突派遣了数万名士兵。巴基斯坦还参与了巴基斯坦西北部的战争（瓦济里斯坦战争）。在美国情报部门的支持下，巴基斯坦正试图从北部部落地区清除塔利班叛乱分子和基地组织成员。
在20世纪70年代，有时是后来，西方和亲西方的政府经常支持有时是初出茅庐的伊斯兰主义者和伊斯兰团体，这些组织后来被视为危险的敌人。西方政

,title,question,answer,context
61244,反恐战争,阿富汗南部的主要战斗部队是什么？,英国第16空袭旅,英国第16空袭旅（后来由皇家海军陆战队增援）与来自澳大利亚、加拿大和荷兰的部队和直升机一起，...


In [69]:
inference_100('阿富汗南部的主要战斗部队是什么？')

1th:  (1, 768)


/root/miniconda3/envs/rag/lib/python3.8/site-packages/transformers/generation/utils.py:2943: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


1th:  (1, 768)

问题：
阿富汗南部的主要战斗部队是什么？

检索结果：
1979，苏联将第四十支军队部署到阿富汗，试图在阿富汗内战中镇压伊斯兰盟军反对伊斯兰主义的政权。这场冲突将土著贫困穆斯林（圣战者）与反宗教的超级大国对立起来，激发了全世界成千上万的穆斯林提供援助，有时甚至为了信仰而战。领导这一泛伊斯兰努力的是巴勒斯坦酋长阿卜杜拉·优素福·阿扎姆。虽然这些“阿富汗阿拉伯人”的军事效力微乎其微，但来自世界各地的大约16000至35000名穆斯林志愿者前来阿富汗作战。
随后，2001年10月，美军（与英国和盟军）入侵阿富汗，驱逐塔利班政权。2001年10月7日，英国和美国军队开始对敌方目标进行空袭。阿富汗首都喀布尔在11月中旬之前已经下跌。剩下的基地组织和塔利班残余势力倒向阿富汗东部崎岖的山区，主要是托拉博拉。去年12月，联军（美国及其盟国）在该地区作战。据信，奥萨马·本·拉登在战斗中逃入巴基斯坦。
在阿富汗，由于政治和部落军阀之间的恶性和破坏性内战，20世纪80年代圣战者战胜苏联并没有带来正义和繁荣，使阿富汗成为世界上最贫穷的国家之一。1992年，由共产党军队统治的阿富汗民主共和国解体，圣战者的民主伊斯兰分子建立了阿富汗伊斯兰国。1996年，一个更加保守和反民主的伊斯兰运动塔利班上台，击败了大部分军阀，占领了大约80%的阿富汗。
奥斯曼帝国的第一个军事单位是奥斯曼一世在13世纪末从居住在安纳托利亚西部山区的部落组织的军队。随着帝国的发展，军事系统成为一个复杂的组织。奥斯曼军队是一个复杂的征兵和封地制度。奥斯曼帝国军队的主要兵团包括Janissary、Sipahi、Akıncı和Mehterın。奥斯曼军队曾经是世界上最先进的战斗力量之一，是最早使用步枪和大炮的军队之一。奥斯曼土耳其人在君士坦丁堡围攻期间开始使用短而宽的猎鹰炮。奥斯曼骑兵依靠高速和机动性，而不是重甲，在快速的土库曼马和阿拉伯马（纯种赛马的祖先）上使用弓箭和短剑，并且经常使用类似于蒙古帝国的战术，例如在新月形的内部包围敌人时假装撤退然后进行真正的攻击。从17世纪中叶到土耳其战争之后，军队的表现明显下降。18世纪在对抗威尼斯的比赛中取得了一些有限的成功，但在北方，欧洲式的俄罗斯军队迫使奥斯曼人让出土地。
当美国在2002年底明确表示要入侵伊拉克时，对美国的支持就降温了。即便如此，许多无条件

,title,question,answer,context
61244,反恐战争,阿富汗南部的主要战斗部队是什么？,英国第16空袭旅,英国第16空袭旅（后来由皇家海军陆战队增援）与来自澳大利亚、加拿大和荷兰的部队和直升机一起，...


### for demo -- only_gen_ckpt5

In [50]:
inference_100('合作区现在有多少名澳门学生？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
合作区现在有多少名澳门学生？

检索结果：
目前，已有230名澳门学生在合作区学习。
横琴粤澳深度合作区成立以来，在合作区就读的澳门学生人数逐年增加。
合作区民生事务局将加强与澳门相关机构的合作，组织澳门居民到合作区实地考察，组织澳门青年到合作区交流，组织澳门高校学生到合作区实习，让更多澳门居民了解合作区的建设发展，吸引他们到横琴居住就业。
在合作区中小学、幼儿园就读的澳门学生享受等同澳门的学费津贴、书簿津贴等福利待遇。
加快实施“民生深度融合工程”方面，明确全力推动澳门新街坊项目年内整体竣工，力争各项澳门标准公共服务和社会福利同步配套到位；落实在合作区就读的澳门学生同等享受澳门本地学生的学费书簿津贴等教育福利待遇；争取在合作区指定医疗机构内使用澳门已上市部分药品及医疗器械；推动在合作区创新创业就业的澳门青年同步享受粤澳两地的扶持政策。

回答：
['230 名 澳 门 学 生']


In [47]:
inference_100('粤港澳大湾区绿色金融联盟包含哪些城市？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
粤港澳大湾区绿色金融联盟包含哪些城市？

检索结果：
人民银行广州分行将继续依托横琴粤澳深度合作区等重大平台，围绕推动大湾区企业赴港澳发行绿色债券、绿色金融标准互认、跨境绿色金融产品创新等重点任务，加强与港澳相关部门的沟通协调，持续推动粤港澳大湾区绿色金融发展。
香港、深圳、广州、澳门四地联合成立了全国首个区域性绿色金融联盟——粤港澳大湾区绿色金融联盟，促进绿色金融发展。
数字经济、绿色金融、文旅产业是国际工商界认为粤港澳大湾区最具成长潜力的领域。
香港作为国际金融中心，可连通国内外绿色和可持续的资金，助力粤港澳大湾区及内地其他省市的绿色转型。
来到粤港澳大湾区，感受到它是整个中国经济最具活力的地方，粤港澳大湾区的数字金融发展机遇挑战和前景是一个非常重要的话题。

回答：
['香 港 、 澳 门 、 深 圳 和 广 州']


In [48]:
inference_100('执行委员会下属有多少个部门？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
执行委员会下属有多少个部门？

检索结果：
合作区执委会是管委会的日常工作机构，其下设的9个职能机构分别依法履行国际推介、招商引资、产业导入、土地开发、项目建设、教育医疗、文化体育、社会保障等相关行政管理和公共服务职能。
鼓励银行在合作区设立科技支行等科技金融专营机构。
规则衔接首先体现在金融交易规则领域，其次体现在开展具体业务相关领域。金融监管的规则衔接也十分关键，对于相同功能或相同法律关系的金融产品，采取一套相同的监管体系尤为重要。
实施细则对多功能自由贸易账户业务定义、市场主体开立条件、业务规则等关键要素加以明确，对业务监管和风险管理安排提出更为细致的要求。
鼓励在合作区设立证券公司、基金公司、再保险公司、银行理财子公司等各类机构。

回答：
['9']


In [49]:
inference_100('执行委员会有哪些职能？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
执行委员会有哪些职能？

检索结果：
合作区执委会是管委会的日常工作机构，其下设的9个职能机构分别依法履行国际推介、招商引资、产业导入、土地开发、项目建设、教育医疗、文化体育、社会保障等相关行政管理和公共服务职能。
合作区执行委员会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
根据合作区发展实际需要，合作区执委会可以自主决定其工作机构聘用的人员及其薪酬标准和福利待遇，建立与绩效目标相适应的薪酬管理制度；同时，合作区执委会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
合作区执行委员会及其工作机构是承担合作区经济、民生管理等相关行政管理和公共服务职能的法定机构。
鼓励银行在合作区设立科技支行等科技金融专营机构。

回答：
['行 政 管 理 和 公 共 服 务']


In [78]:
inference_100('执行委员会有哪些职能？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
执行委员会有哪些职能？

检索结果：
合作区执委会是管委会的日常工作机构，其下设的9个职能机构分别依法履行国际推介、招商引资、产业导入、土地开发、项目建设、教育医疗、文化体育、社会保障等相关行政管理和公共服务职能。
合作区执行委员会及其工作机构是承担合作区经济、民生管理等相关行政管理和公共服务职能的法定机构。
合作区执行委员会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
鼓励银行在合作区设立科技支行等科技金融专营机构。
根据合作区发展实际需要，合作区执委会可以自主决定其工作机构聘用的人员及其薪酬标准和福利待遇，建立与绩效目标相适应的薪酬管理制度；同时，合作区执委会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。

回答：
['行 政 管 理 和 公 共 服 务']


In [33]:
inference_100('横琴有哪些学校？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
横琴有哪些学校？

检索结果：
教育资源方面，横琴已有澳门大学、横琴容闳公学、哈罗礼德学校、德威国际高中、横琴一中、横琴新区第一小学、首都师范大学横琴子期实验小学、首都师范大学横琴伯牙小学、子期中学（建设中）等学校，从基础教育到国际化视野，优质教育资源充沛输出。
医疗配套方面，珠海人民医院横琴分院、横琴至和国际生命科学中心（在建）、横琴中心医院（在建）三座三甲医院也逐渐落地。
横琴自贸区属于全国人才管理改革示范区，根据区内特殊人才实际贡献，在签证、居留、居住、医疗、配偶就业、子女入学、住房补贴等方面给予相应奖励，对符合条件的大学生落户横琴新区。
首创“公办民管”与“高校联姻办学”办学模式，引进珠海华发教育、全国“双一流”首都师范大学等优质教育品牌为横琴教育助力。
横琴哈罗礼德学校、横琴华发容闳学校、珠海市华德教育培训有限公司（珠海德威高中）等高端民办机构，提供国际化、多元化教育环境。

回答：
['澳 门 大 学 、 横 琴 容 纳 公 学 、 哈 罗 礼 德 学 校 、 德 威 国 际 学 校 和 横 琴 一 中']


In [77]:
inference_100('横琴有哪些学校？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
横琴有哪些学校？

检索结果：
教育资源方面，横琴已有澳门大学、横琴容闳公学、哈罗礼德学校、德威国际高中、横琴一中、横琴新区第一小学、首都师范大学横琴子期实验小学、首都师范大学横琴伯牙小学、子期中学（建设中）等学校，从基础教育到国际化视野，优质教育资源充沛输出。
研究在横琴设立澳门子弟学校。
来自不同地方的学子们在横琴相遇相知，互助互融，为合作区教育多元发展注入活力。
横琴各学校努力打造书香校园，建设环境一流的图书馆阅读空间，为提供品质阅读服务、提升学校文化品位打下良好基础。
横琴自贸区属于全国人才管理改革示范区，根据区内特殊人才实际贡献，在签证、居留、居住、医疗、配偶就业、子女入学、住房补贴等方面给予相应奖励，对符合条件的大学生落户横琴新区。

回答：
['澳 门 大 学 、 横 琴 容 闳 公 学 、 哈 罗 礼 德 学 校 、 德 威 国 际 高 中']


### Shallow train, 2poch

In [22]:
inference_100('执行委员会有哪些职能？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
执行委员会有哪些职能？

检索结果：
合作区执委会是管委会的日常工作机构，其下设的9个职能机构分别依法履行国际推介、招商引资、产业导入、土地开发、项目建设、教育医疗、文化体育、社会保障等相关行政管理和公共服务职能。
合作区执行委员会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
根据合作区发展实际需要，合作区执委会可以自主决定其工作机构聘用的人员及其薪酬标准和福利待遇，建立与绩效目标相适应的薪酬管理制度；同时，合作区执委会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
合作区执行委员会及其工作机构是承担合作区经济、民生管理等相关行政管理和公共服务职能的法定机构。
做好就业帮扶工作，通过举办劳务招聘会、发布用工信息等方式，动员合作区企业优先安排正安县劳务人员就业；回应企业需求，按照企业提供的就业信息，探索开展订单式培训班，并将劳动力整体输送至相关企业。

回答：
['合 作 区 执 行 委 员 会']


In [17]:
inference_100('粤港澳大湾区绿色金融联盟包含哪些城市？',5)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1th:  (1, 768)
1th:  (1, 768)

问题：
粤港澳大湾区绿色金融联盟包含哪些城市？

检索结果：
人民银行广州分行将继续依托横琴粤澳深度合作区等重大平台，围绕推动大湾区企业赴港澳发行绿色债券、绿色金融标准互认、跨境绿色金融产品创新等重点任务，加强与港澳相关部门的沟通协调，持续推动粤港澳大湾区绿色金融发展。
香港、深圳、广州、澳门四地联合成立了全国首个区域性绿色金融联盟——粤港澳大湾区绿色金融联盟，促进绿色金融发展。
数字经济、绿色金融、文旅产业是国际工商界认为粤港澳大湾区最具成长潜力的领域。
香港作为国际金融中心，可连通国内外绿色和可持续的资金，助力粤港澳大湾区及内地其他省市的绿色转型。
广东省地方金融监督管理局局长于海平视频参会时说，过去两年在中央有关部门的指导支持下，粤港澳三地密切合作，围绕绿色金融发展和创新进行了一系列探索，在绿色金融基础设施、绿色金融业务监管等方面形成合力，推动粤港澳大湾区绿色金融合作迈上新的台阶。

回答：
['香 港 和 澳 门']


### K=5

In [38]:
inference_100('合作区现有多少名学生？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
合作区现有多少名学生？

检索结果：
合作区现有12所教育机构，其中8所公办学校，4所民办教育机构，搭建合作区多元教育平台。
目前，已有230名澳门学生在合作区学习。
合作区民生事务局将继续为企业与求职者搭桥引线，组织优质企业到各高校以及澳门进行招聘，为合作区建设匹配优秀人才。
合作区民生事务局局长黄宇杰致辞时表示，希望学生们通过考察交流，进一步了解合作区的发展战略及未来规划，同时将自身职涯规划与横琴四大产业发展方向相结合，积极参与合作区的建设发展。
合作区普及学前教育，大力发展普惠性学前教育，加强优质公办学位建设，支持社会力量举办民办幼儿园。

回答：
['12']


In [39]:
inference_100('合作区现有多少名澳门学生？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
合作区现有多少名澳门学生？

检索结果：
目前，已有230名澳门学生在合作区学习。
横琴粤澳深度合作区成立以来，在合作区就读的澳门学生人数逐年增加。
合作区民生事务局将加强与澳门相关机构的合作，组织澳门居民到合作区实地考察，组织澳门青年到合作区交流，组织澳门高校学生到合作区实习，让更多澳门居民了解合作区的建设发展，吸引他们到横琴居住就业。
在合作区中小学、幼儿园就读的澳门学生享受等同澳门的学费津贴、书簿津贴等福利待遇。
加快实施“民生深度融合工程”方面，明确全力推动澳门新街坊项目年内整体竣工，力争各项澳门标准公共服务和社会福利同步配套到位；落实在合作区就读的澳门学生同等享受澳门本地学生的学费书簿津贴等教育福利待遇；争取在合作区指定医疗机构内使用澳门已上市部分药品及医疗器械；推动在合作区创新创业就业的澳门青年同步享受粤澳两地的扶持政策。

回答：
['230']


In [18]:
inference_100('绿色金融联盟包含哪些城市？',5)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1th:  (1, 768)
1th:  (1, 768)


['香 港 、 澳 门 、 新 西 兰 、 台 湾 、 澳 大 利 亚 和 台 湾']

In [32]:
inference_100('粤港澳大湾区绿色金融联盟包含哪些城市？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
粤港澳大湾区绿色金融联盟包含哪些城市？

检索结果：
人民银行广州分行将继续依托横琴粤澳深度合作区等重大平台，围绕推动大湾区企业赴港澳发行绿色债券、绿色金融标准互认、跨境绿色金融产品创新等重点任务，加强与港澳相关部门的沟通协调，持续推动粤港澳大湾区绿色金融发展。
香港、深圳、广州、澳门四地联合成立了全国首个区域性绿色金融联盟——粤港澳大湾区绿色金融联盟，促进绿色金融发展。
数字经济、绿色金融、文旅产业是国际工商界认为粤港澳大湾区最具成长潜力的领域。
香港作为国际金融中心，可连通国内外绿色和可持续的资金，助力粤港澳大湾区及内地其他省市的绿色转型。
来到粤港澳大湾区，感受到它是整个中国经济最具活力的地方，粤港澳大湾区的数字金融发展机遇挑战和前景是一个非常重要的话题。

回答：
['香 港 、 澳 门 、 深 圳 和 广 州']


In [33]:
inference_100('执行委员会下属有几个工作机构？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
执行委员会下属有几个工作机构？

检索结果：
合作区执委会是管委会的日常工作机构，其下设的9个职能机构分别依法履行国际推介、招商引资、产业导入、土地开发、项目建设、教育医疗、文化体育、社会保障等相关行政管理和公共服务职能。
鼓励银行在合作区设立科技支行等科技金融专营机构。
根据合作区发展实际需要，合作区执委会可以自主决定其工作机构聘用的人员及其薪酬标准和福利待遇，建立与绩效目标相适应的薪酬管理制度；同时，合作区执委会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
合作区执行委员会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
合作区执行委员会及其工作机构是承担合作区经济、民生管理等相关行政管理和公共服务职能的法定机构。

回答：
['9']


In [60]:
inference_100('执行委员会下属有几个部门？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
执行委员会下属有几个部门？

检索结果：
合作区执委会是管委会的日常工作机构，其下设的9个职能机构分别依法履行国际推介、招商引资、产业导入、土地开发、项目建设、教育医疗、文化体育、社会保障等相关行政管理和公共服务职能。
鼓励银行在合作区设立科技支行等科技金融专营机构。
规则衔接首先体现在金融交易规则领域，其次体现在开展具体业务相关领域。金融监管的规则衔接也十分关键，对于相同功能或相同法律关系的金融产品，采取一套相同的监管体系尤为重要。
实施细则对多功能自由贸易账户业务定义、市场主体开立条件、业务规则等关键要素加以明确，对业务监管和风险管理安排提出更为细致的要求。
横琴建行托管品种已扩充至八大类，分别为私募股权基金、信托计划、保债计划、不动产投资信托基金、合格境外有限合伙人、合格境内有限合伙人、合格境外机构投资者和境外机构投资银行间债券市场等。

回答：
['9']


In [34]:
inference_100('执行委员会有哪些职能？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
执行委员会有哪些职能？

检索结果：
合作区执委会是管委会的日常工作机构，其下设的9个职能机构分别依法履行国际推介、招商引资、产业导入、土地开发、项目建设、教育医疗、文化体育、社会保障等相关行政管理和公共服务职能。
合作区执行委员会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
根据合作区发展实际需要，合作区执委会可以自主决定其工作机构聘用的人员及其薪酬标准和福利待遇，建立与绩效目标相适应的薪酬管理制度；同时，合作区执委会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
合作区执行委员会及其工作机构是承担合作区经济、民生管理等相关行政管理和公共服务职能的法定机构。
鼓励银行在合作区设立科技支行等科技金融专营机构。

回答：
['行 政 管 理 和 公 共 服 务']


In [39]:
inference_100('电子围网有什么用处?',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
电子围网有什么用处?

检索结果：
积极探索支持 " 横琴金融 30 条 " 创新举措落地实施，重点聚焦合作区资金 " 电子围网 "，配合监管机构，尽快推动电子围网相关业务落地，并在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试，助推合作区跨境金融服务向纵深发展。
完善开放条件下的金融监管机制，创新基于电子围网系统的跨境金融管理体系，实现跨境资金“管得住、放得开”。
探索构建电子围网系统。
“横琴金融30条”强调金融市场和金融基础设施实现互联互通，建立电子围网，扩大跨境资产转让范围，推动跨境人民币结算等，保障跨境金融工作平稳有序开展。
“横琴金融30条”提出，加强合作区金融市场与澳门、香港离岸金融市场的联动、稳步扩大跨境资产转让范围、稳步开展“跨境理财通”业务试点等具体举措，同时还创新性提出优化改造自由贸易账户（FT账户）系统，通过金融账户隔离，在合作区建立资金电子围网。

回答：
['跨 境 金 融 服 务']


In [57]:
inference_100('横琴三十条讲了什么？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
横琴三十条讲了什么？

检索结果：
关于药学技术人员在横琴药品零售单位执业的规定减少了以往较为繁复的执业手续，“法规更加齐全、配套更为完善，从而花更少时间、更少精力落地横琴。”
横琴青少年运动员群情激昂，纷纷表示不仅学会了如何在运动中保护自己，也增强了他们的伤病防范意识，调动了他们参加训练的积极性和主动性，激起了他们为横琴争夺荣誉的信心和决心。
教育资源方面，横琴已有澳门大学、横琴容闳公学、哈罗礼德学校、德威国际高中、横琴一中、横琴新区第一小学、首都师范大学横琴子期实验小学、首都师范大学横琴伯牙小学、子期中学（建设中）等学校，从基础教育到国际化视野，优质教育资源充沛输出。
“横琴之书”讲述横琴与澳门交往交流、交心交融的文化故事，带领市民游客回顾横琴的建设历史，探索粤澳深度合作的规划蓝图，以及领略琴澳创新融合的发展成果。
优化完善体制机制，按照“应放尽放”“能放尽放”原则再向横琴合作区下放一批省级行政管理职权，加强横琴合作区干部队伍建设，吸引更多优秀人才到横琴合作区干事创业。

回答：
['横 琴 三 十 条']


In [59]:
inference_100('横琴已有哪些教育资源？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
横琴已有哪些教育资源？

检索结果：
教育资源方面，横琴已有澳门大学、横琴容闳公学、哈罗礼德学校、德威国际高中、横琴一中、横琴新区第一小学、首都师范大学横琴子期实验小学、首都师范大学横琴伯牙小学、子期中学（建设中）等学校，从基础教育到国际化视野，优质教育资源充沛输出。
横琴自贸区属于全国人才管理改革示范区，根据区内特殊人才实际贡献，在签证、居留、居住、医疗、配偶就业、子女入学、住房补贴等方面给予相应奖励，对符合条件的大学生落户横琴新区。
作区将继续把首都基础教育的引领示范作用与横琴合作区当地经济社会发展紧密结合，促进琴澳优势教育资源互补与双赢，助力横琴打造便利澳门居民生活就业新家园。
横琴各学校努力打造书香校园，建设环境一流的图书馆阅读空间，为提供品质阅读服务、提升学校文化品位打下良好基础。
横琴多年来不断推动体育事业高质量发展，各类体育场馆和设施不断完善，更有珠海WTA超级精英赛、珠海网球冠军赛（ATP250）、横琴马拉松等大型赛事IP，已经形成浓厚的体育产业发展氛围，助力横琴国际休闲旅游岛建设，不仅为横琴、珠海、澳门的各类体育协会和行业企业提供共谋发展的平台，更为广大体育爱好者提供了一个相互学习、相互交流、相互促进的大舞台。

回答：
['澳 大 利 亚 、 新 西 兰 、 加 拿 大 、 英 国 、 法 国 、 美 国']


In [61]:
inference_100('合作区有哪些项目在运行中？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
合作区有哪些项目在运行中？

检索结果：
合作区体育联合会会长单位负责人吕品德表示，联合会将在现有体育资源基础上，统筹协调好合作区内的各类体育协会和企业、体育场地资源，组织开展多元化、有特色的赛事活动，统筹做好各类体育培训工作，搭建数字化体育平台，整合体育场馆和设施的实时数据，为工作和生活在合作区的琴澳居民提供更优质的体育服务。
合作区民生事务局将在劳务协作方面持续发力，建设好“一县一企”农村劳动力稳岗就业基地，不断完善两地劳务协作常态化机制，积极推进劳务协作的深度合作，引导合作区优质企业开展帮扶对接，对符合条件的帮扶企业发放政策性补贴，为奋力推动东西部协作实现高质量发展贡献琴澳力量，努力打造民生合作领域创新典范。
支持外资和社会资本参与合作区重点产业建设，进一步推动私募股权投资基金跨境投资业务，鼓励更多符合条件的资产管理机构参与合作区合格境外有限合伙人（QFLP）试点和合格境内有限合伙人（QDLP）试点。
《意见》提出，支持实体方面，加强对文旅、商贸、会展产业的金融支持，完善创业金融服务，促进国际融资租赁和商业保理业务发展等。未来，合作区金融服务能力将不断提升，配合合作区产业的多元发展。
经济、金融领域专家及业界代表围绕合作区金融业未来发展重点等方向，纷纷为合作区建言献策。

回答：
['金 融 、 保 险 、 房 地 产 、 医 疗 、 教 育 、 文 化 和 旅 游 业']


In [80]:
inference_100('合作区有哪些项目在运行中？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
合作区有哪些项目在运行中？

检索结果：
合作区将继续举办足球、篮球等项目校际联赛，全面营造校园体育文化氛围，完善青少年体育赛事体系，形成全方位、多元化竞赛格局，打造合作区特色校园体育发展模式。
自合作区揭牌以来，合作区与珠海开展务实合作，大力推动跨区基础设施互联互通，进一步增强合作区与珠海之间的交通联系、经济往来、人文交流、互利合作。
经济、金融领域专家及业界代表围绕合作区金融业未来发展重点等方向，纷纷为合作区建言献策。
合作区体育联合会的成立对进一步繁荣发展合作区文体事业、推进全民健身运动将发挥十分重要的作用，希望联合会充分发挥连接政府与体育类企业的纽带和桥梁作用，促进合作区内的体育类企业建立高标准的管理与服务体系，同时积极联动粤港澳大湾区各城市，推进各项体育赛事活动在横琴落地开花，助力琴澳体育融合发展。
支持外资和社会资本参与合作区重点产业建设，进一步推动私募股权投资基金跨境投资业务，鼓励更多符合条件的资产管理机构参与合作区合格境外有限合伙人（QFLP）试点和合格境内有限合伙人（QDLP）试点。

回答：
['足 球 、 篮 球 、 羽 毛 球 和 乒 乓 球']


In [85]:
inference_100('有求职需求，可以访问什么网址查看？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
有求职需求，可以访问什么网址查看？

检索结果：
现在合作区居民通过粤澳跨境数据验证平台，5分钟可以搞定跨境资信证明。
为满足澳门居民求职需求，合作区民生事务局携手澳门劳工事务局已于2022年推出“横琴·澳门居民招聘专区”平台（https://zhaopin.hq-hr.com），为澳门青年提供便利的求职渠道和多元化就业岗位。
横琴税务局工作人员对市场主体关心关注的政策要点开展了详细介绍，并就企业实质性运营、个人所得税享惠条件等热点问题进行解答。
通过“跨境通办、一地两注”机制办理跨境商事登记时间缩减60%。
指导广东省银行外汇和跨境人民币业务展业自律机制发布业务政策问答，明确合作区银行可以根据境外机构需要，将境外机构人民币银行结算账户（人民币NRA账户）内的资金转存为定期存款以及用于购买大额存单

回答：
['横 琴 · 澳 门 居 民 招 聘 专 区']


### K=10

In [47]:
inference_100('电子围网有什么用处?',10)

1th:  (1, 768)
1th:  (1, 768)

问题：
电子围网有什么用处?

检索结果：
积极探索支持 " 横琴金融 30 条 " 创新举措落地实施，重点聚焦合作区资金 " 电子围网 "，配合监管机构，尽快推动电子围网相关业务落地，并在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试，助推合作区跨境金融服务向纵深发展。
完善开放条件下的金融监管机制，创新基于电子围网系统的跨境金融管理体系，实现跨境资金“管得住、放得开”。
探索构建电子围网系统。
“横琴金融30条”强调金融市场和金融基础设施实现互联互通，建立电子围网，扩大跨境资产转让范围，推动跨境人民币结算等，保障跨境金融工作平稳有序开展。
“横琴金融30条”提出，加强合作区金融市场与澳门、香港离岸金融市场的联动、稳步扩大跨境资产转让范围、稳步开展“跨境理财通”业务试点等具体举措，同时还创新性提出优化改造自由贸易账户（FT账户）系统，通过金融账户隔离，在合作区建立资金电子围网。
支持合作区构建特色微电子产业链，建设集成电路先进测试技术和服务平台，布局芯片研发和制造项目，建设全球电子元器件集散中心。
细化落实《横琴意见》措施。其中，以澳门新街坊等民生项目为重点，积极推动惠民生、暖民心举措优先落地实施；注重推进基础性、先导性工作，做好基于现有自由贸易账户（FT账户）体系改造，推动资金“电子围网”落地实施；在与澳门金融市场互联互通、人民币跨境使用、跨境资金流动等领域落实开放措施，提升琴澳金融一体化水平。
持合作区、澳门相关高等学校、科研机构在确保个人信息和重要数据安全前提下，实现科学研究数据依法跨境互联互通。
按照横琴“立足服务澳门、琴澳一体化”的战略定位，设计更加开放的金融制度安排，通过构建电子围网系统，创新跨境金融管理，探索跨境资金自由流动途径。
合作区着力构建与澳门一体化高水平开放的新体系，实现货物“一线”放开，“二线”管住，除国家法律、行政法规明确规定不予免（保）税的货物及物品外，其他货物及物品免（保）税进入，人员进出高度便利，创新跨境金融管理，建立高度便利的市场准入制度，促进国际互联网数据跨境安全有序流动，实现与澳门人流、物流、资金流、信息流等高效便捷流动。

回答：
['跨 境 电 子 围 网 系 统']


In [52]:
inference_100('建立电子围网有什么积极的作用?',10)

1th:  (1, 768)
1th:  (1, 768)

问题：
建立电子围网有什么积极的作用?

检索结果：
积极探索支持 " 横琴金融 30 条 " 创新举措落地实施，重点聚焦合作区资金 " 电子围网 "，配合监管机构，尽快推动电子围网相关业务落地，并在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试，助推合作区跨境金融服务向纵深发展。
探索构建电子围网系统。
完善开放条件下的金融监管机制，创新基于电子围网系统的跨境金融管理体系，实现跨境资金“管得住、放得开”。
“横琴金融30条”强调金融市场和金融基础设施实现互联互通，建立电子围网，扩大跨境资产转让范围，推动跨境人民币结算等，保障跨境金融工作平稳有序开展。
支持合作区构建特色微电子产业链，建设集成电路先进测试技术和服务平台，布局芯片研发和制造项目，建设全球电子元器件集散中心。
“横琴金融30条”提出，加强合作区金融市场与澳门、香港离岸金融市场的联动、稳步扩大跨境资产转让范围、稳步开展“跨境理财通”业务试点等具体举措，同时还创新性提出优化改造自由贸易账户（FT账户）系统，通过金融账户隔离，在合作区建立资金电子围网。
细化落实《横琴意见》措施。其中，以澳门新街坊等民生项目为重点，积极推动惠民生、暖民心举措优先落地实施；注重推进基础性、先导性工作，做好基于现有自由贸易账户（FT账户）体系改造，推动资金“电子围网”落地实施；在与澳门金融市场互联互通、人民币跨境使用、跨境资金流动等领域落实开放措施，提升琴澳金融一体化水平。
促进合作区与澳门一体化发展，对标国际标准制定账户规则，通过多功能自由贸易账户构建资金“电子围网”，在合作区探索跨境资金自由流入流出和推进资本项目可兑换。
持合作区、澳门相关高等学校、科研机构在确保个人信息和重要数据安全前提下，实现科学研究数据依法跨境互联互通。
“横琴金融30条”是横琴粤澳深度合作区成立以来，由中央部委会同广东省政府发布的首个金融产业创新发展专项政策。此外，还在全国首次推出了一些制度安排，如“构建资金电子围网系统、在合作区探索允许澳门元作为小额支付使用”等，这极大地促进了澳门深度融入国家发展大局。

回答：
['资 金 电 子 围 网']


In [79]:
inference_100('建立电子围网有什么积极的作用?',10)

1th:  (1, 768)
1th:  (1, 768)

问题：
建立电子围网有什么积极的作用?

检索结果：
探索构建电子围网系统。
积极探索支持 " 横琴金融 30 条 " 创新举措落地实施，重点聚焦合作区资金 " 电子围网 "，配合监管机构，尽快推动电子围网相关业务落地，并在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试，助推合作区跨境金融服务向纵深发展。
完善开放条件下的金融监管机制，创新基于电子围网系统的跨境金融管理体系，实现跨境资金“管得住、放得开”。
“横琴金融30条”强调金融市场和金融基础设施实现互联互通，建立电子围网，扩大跨境资产转让范围，推动跨境人民币结算等，保障跨境金融工作平稳有序开展。
强合作区金融市场与澳门、香港离岸金融市场的联动，探索构建电子围网系统，推动合作区金融市场率先高度开放。
优化改造自由贸易账号系统，通过金融账户隔离，在合作区建立资金电子围网。
按照横琴“立足服务澳门、琴澳一体化”的战略定位，设计更加开放的金融制度安排，通过构建电子围网系统，创新跨境金融管理，探索跨境资金自由流动途径。
支持合作区构建特色微电子产业链，建设集成电路先进测试技术和服务平台，布局芯片研发和制造项目，建设全球电子元器件集散中心。
《横琴粤澳深度合作区建设总体方案》提出“探索构建电子围网系统，推动合作区金融市场率先高度开放”、“在合作区内探索跨境资本自由流入流出和推进资本项目可兑换”等一系列政策安排，为合作区发展率先高度开放的金融产业提供了政策支撑。
“横琴金融30条”首次提出在合作区探索允许澳门元作为小额支付使用、构建资金电子围网系统等多项政策，将有力助推合作区成为国家金融改革创新试验田。

回答：
['金 融 2 / 探 索 构 建 电 子 围 网 系 统']


In [48]:
inference_100('执行委员会有哪些职能？',10)

1th:  (1, 768)
1th:  (1, 768)

问题：
执行委员会有哪些职能？

检索结果：
合作区执委会是管委会的日常工作机构，其下设的9个职能机构分别依法履行国际推介、招商引资、产业导入、土地开发、项目建设、教育医疗、文化体育、社会保障等相关行政管理和公共服务职能。
合作区执行委员会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
根据合作区发展实际需要，合作区执委会可以自主决定其工作机构聘用的人员及其薪酬标准和福利待遇，建立与绩效目标相适应的薪酬管理制度；同时，合作区执委会可以从境内外专业人士中选聘工作人员，并应当与聘用的人员签订劳动合同。
合作区执行委员会及其工作机构是承担合作区经济、民生管理等相关行政管理和公共服务职能的法定机构。
鼓励银行在合作区设立科技支行等科技金融专营机构。
做好就业帮扶工作，通过举办劳务招聘会、发布用工信息等方式，动员合作区企业优先安排正安县劳务人员就业；回应企业需求，按照企业提供的就业信息，探索开展订单式培训班，并将劳动力整体输送至相关企业。
根据合作区发展实际需要，合作区执行委员会可以自主决定其工作机构聘用的人员及其薪酬标准和福利待遇，建立与绩效目标相适应的薪酬管理制度。
两项新的法规让跨境执业准入条件更简便优化，也拓展了相关人员的执业范围。
励合作区仲裁机构借鉴国际商事仲裁机构先进的管理体制和管理模式，建立国际通行商事仲裁机制，搭建琴澳仲裁合作平台。
合作区公安机关出入境管理机构可以受理、审批签发并制作外国人签证和停居留证件。对在合作区企业、高等学校、科研机构工作的符合条件的内地人员实行商务和人才签注备案制。

回答：
['国 际 货 币 基 金 组 织']


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/bart-fnlp-chinese")
# print(tokenizer.model_max_length)


In [4]:
print(tokenizer.model_max_length)

1000000000000000019884624838656


# Juncong test

In [89]:
inference_100('澳门居民有求职需求，可以访问什么网址查看？',5)

1th:  (1, 768)
1th:  (1, 768)

问题：
澳门居民有求职需求，可以访问什么网址查看？

检索结果：
为满足澳门居民求职需求，合作区民生事务局携手澳门劳工事务局已于2022年推出“横琴·澳门居民招聘专区”平台（https://zhaopin.hq-hr.com），为澳门青年提供便利的求职渠道和多元化就业岗位。
持有效正式澳门驾驶执照的澳门永久性居民，持正式澳门驾驶执照，以及身份证明原件、身体条件证明、照片，可以直接免试换领内地驾驶证，无需参加考试。
港澳居民个人可通过资金多功能自由贸易（电子围网）账户办理工作生活方面的资金结算，便利澳门居民生活就业的金融环境，满足港澳居民在合作区日常工作生活的需求。
现在合作区居民通过粤澳跨境数据验证平台，5分钟可以搞定跨境资信证明。
中国工商银行的“琴澳社保通”对澳门居民服务便利化再度升级，社保跨境办流程更加完善，澳门居民可以实现珠海社保卡“澳门办澳门领”。

回答：
['横 琴 · 澳 门 居 民 招 聘 专 区']


In [90]:
inference_100('粤港澳要实现金融融通，目前还存在哪些问题？')

1th:  (1, 768)
1th:  (1, 768)

问题：
粤港澳要实现金融融通，目前还存在哪些问题？

检索结果：
粤港澳实现金融融通，将减少粤港澳大湾区的资金流通难度，增加货币管理弹性，但也带来了新的金融风险。
扩大开放的同时，建立和完善金融风险监测、防范和化解体系，加大对跨境资金异常流动的监测力度，加强粤港澳反洗钱、反恐怖融资和反逃税的监管合作，加强粤港澳金融监管合作，提升打击跨境洗钱和非法金融活动的有效性。
随着国家建设“一带一路”倡议的深入实施，粤港两地金融界把握机遇，在CEPA框架下加快实施粤港金融服务贸易自由化，以广东自贸区为纽带，促进两地金融市场的连接与融合，推动粤港金融合作取得了一系列的新成果。
金融服务“互联互通”不够。不同金融体系拥有不同的制度、法律、政策等安排，粤港澳三地仍面临如何接轨的问题。
交通银行广东省分行积极找寻国家政策和业务拓展之间的链接点和发力点，依托港澳，服务内地，在服务实体经济、金融改革实践、三地市场互联互通等方面持续深化粤港澳金融合作。

回答：
['货 币 管 理 弹 性']


In [91]:
inference_100('“三升三降”具体讲的是哪些内容？')

1th:  (1, 768)
1th:  (1, 768)

问题：
“三升三降”具体讲的是哪些内容？

检索结果：
横琴与澳门，拥有“两种社会制度、两个关税区、两种税制体系、两种货币”的特殊差异。
制度创新，引领示范。
《横琴意见》和《前海意见》在“远近结合、民生优先、循序渐进、防控风险”的工作思路下，围绕经济高质量发展的需求，聚焦金融改革的重点领域，在两地各部署了五大方面30项金融改革创新工作。
横琴税务局工作人员对市场主体关心关注的政策要点开展了详细介绍，并就企业实质性运营、个人所得税享惠条件等热点问题进行解答。
“横琴金融30条”围绕六个方面提出多条具体措施，是从规划走向落实的关键性一步，将打通金融行业原有的堵点和痛点，为澳门多元化经济带来重大发展机遇。

回答：
['三 升 三 降']


In [97]:
inference_100('在数字金融方面')

1th:  (1, 768)
1th:  (1, 768)

问题：
在数字金融方面

检索结果：
加快金融大数据建设，为数字金融促进创新提供数据基础；推进数字新基建，为数字金融促进产业金融提供基础设施；清除一些政策障碍，让数字金融更好地支持跨境交易，促进粤港澳融合。
银行创新科技型中小企业信贷体系，有利于为数字技术企业提供专业的金融服务。
金融科技积极服务实体经济：近年来，科技力量成为金融行业数字化转型的主要力量之一，也成为驱动金融业创新发展的主要变革抓手。金融科技在提升金融的普惠性、赋能实体经济的高质量发展方面扮演着越来越重要的角色。
在数字金融方面，《报告》称大湾区也有非常突出的金融科技发展实践，可以总结为 “三升三降”：扩大服务规模、提高业务效率、改善用户体验、降低运营成本、控制业务风险、减少人工接触。展望未来，数字金融在助力大湾区进一步融合发展方面具有良好的前景。
加大金融服务实体经济发展力度

回答：
['跨 境 交 易']


In [98]:
inference_100('总结下《横琴意见》文件讲了什么内容')

1th:  (1, 768)
1th:  (1, 768)

问题：
总结下《横琴意见》文件讲了什么内容

检索结果：
《横琴意见》为横琴合作区深化金融改革创新，推进澳琴一体化，进而促进澳门经济适度多元发展提供了坚实的政策支持。
“横琴之书”讲述横琴与澳门交往交流、交心交融的文化故事，带领市民游客回顾横琴的建设历史，探索粤澳深度合作的规划蓝图，以及领略琴澳创新融合的发展成果。
《横琴意见》和《前海意见》在“远近结合、民生优先、循序渐进、防控风险”的工作思路下，围绕经济高质量发展的需求，聚焦金融改革的重点领域，在两地各部署了五大方面30项金融改革创新工作。
横琴青少年运动员群情激昂，纷纷表示不仅学会了如何在运动中保护自己，也增强了他们的伤病防范意识，调动了他们参加训练的积极性和主动性，激起了他们为横琴争夺荣誉的信心和决心。
横琴税务局工作人员对市场主体关心关注的政策要点开展了详细介绍，并就企业实质性运营、个人所得税享惠条件等热点问题进行解答。

回答：
['横 琴 合 作 区 深 化 金 融 改 革 创 新 ， 推 进 澳 琴 一 体 化']


In [107]:
inference_100('电子围网系统建立')

1th:  (1, 768)
1th:  (1, 768)

问题：
电子围网系统建立

检索结果：
探索构建电子围网系统。
积极探索支持 " 横琴金融 30 条 " 创新举措落地实施，重点聚焦合作区资金 " 电子围网 "，配合监管机构，尽快推动电子围网相关业务落地，并在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试，助推合作区跨境金融服务向纵深发展。
完善开放条件下的金融监管机制，创新基于电子围网系统的跨境金融管理体系，实现跨境资金“管得住、放得开”。
优化改造自由贸易账号系统，通过金融账户隔离，在合作区建立资金电子围网。
按照横琴“立足服务澳门、琴澳一体化”的战略定位，设计更加开放的金融制度安排，通过构建电子围网系统，创新跨境金融管理，探索跨境资金自由流动途径。

回答：
['探 索 构 建 电 子 围 网 系 统']


In [108]:
inference_100('横琴金融30条有哪些内容？有什么积极作用？')

1th:  (1, 768)
1th:  (1, 768)

问题：
横琴金融30条有哪些内容？有什么积极作用？

检索结果：
“横琴金融30条”出台后可以预见一个更开放的金融市场环境即将到来，不论是金融从业者还是金融机构都能扩展自身的发展空间。
“横琴金融30条”中重要金融改革举措正加速落地。
金融30条’支持深合区建设的意见，明确了未来横琴金融发展的一个方向。横琴金融市场和澳门金融市场的将会进一步流动。
“横琴金融30条”“前海金融30条”围绕粤港澳跨境金融合作，在金融机构合作、金融市场合作等方面进行了一系列制度性安排。
“横琴金融30条”使金融机构进入横琴的门槛降低，“横琴金融30条”允许澳门金融机构在横琴设立分支机构，促使其更加便利参与内地市场，一方面促进现代金融业的发展，另一方面助力澳门经济发展多元化。

回答：
['扩 展 自 身 的 发 展 空 间']


In [109]:
inference_100('粤港澳大湾区绿色金融联盟包含哪几个地区')

1th:  (1, 768)
1th:  (1, 768)

问题：
粤港澳大湾区绿色金融联盟包含哪几个地区

检索结果：
香港、深圳、广州、澳门四地联合成立了全国首个区域性绿色金融联盟——粤港澳大湾区绿色金融联盟，促进绿色金融发展。
人民银行广州分行将继续依托横琴粤澳深度合作区等重大平台，围绕推动大湾区企业赴港澳发行绿色债券、绿色金融标准互认、跨境绿色金融产品创新等重点任务，加强与港澳相关部门的沟通协调，持续推动粤港澳大湾区绿色金融发展。
广东省地方金融监督管理局局长于海平视频参会时说，过去两年在中央有关部门的指导支持下，粤港澳三地密切合作，围绕绿色金融发展和创新进行了一系列探索，在绿色金融基础设施、绿色金融业务监管等方面形成合力，推动粤港澳大湾区绿色金融合作迈上新的台阶。
粤港澳大湾区各地陆续出台绿色金融政策，明确发展重点和进度。
中国人民银行研究局局长王信在视频致辞时表示，整体看粤港澳大湾区绿色金融发展前景非常广阔，期待与大湾区绿色金融联盟在共同创新绿色金融产品、应对金融风险等方面加强交流合作，推动中国经济绿色低碳转型发展。

回答：
['香 港 、 深 圳 、 广 州 、 澳 门']


In [110]:
inference_100('澳门最大的居委会是？')

1th:  (1, 768)
1th:  (1, 768)

问题：
澳门最大的居委会是？

检索结果：
被称为澳门最大“居委会”的澳门街坊会联合总会，早在2019年就进驻横琴，通过组建专业的工作坊与小组，积极开展各类活动，加速琴澳两地居民共融。
“澳门新街坊”项目是粤澳两地合作为澳门居民建设的首个集居住、教育、卫生、社会服务于一体的综合民生工程，预计提供约4000个住宅单位，建成后将为超过1万名澳门居民提供优质居住空间。
近年来，合作区围绕澳门居民需求，以建设首个标志性综合民生项目“澳门新街坊”为切入点，深度对接澳门公共服务和社会保证体系，便利澳门居民生活就业新空间正加速形成。
完善澳门居民在合作区生活、就业、创业制度保障的重要举措，有助于推动澳门长期繁荣稳定、更好融入国家发展大局。
支持合作区建立与澳门社会服务合作机制，促进两地社区治理和服务融合发展。

回答：
['澳 门 街 坊 会 联 合 总 会']


In [114]:
inference_100('《横琴粤澳深度合作区发展促进条例》',15)

1th:  (1, 768)
1th:  (1, 768)

问题：
《横琴粤澳深度合作区发展促进条例》

检索结果：
制度保障方面，五个金融政策正式实施（即《横琴粤澳深度合作区外商投资股权投资类企业试点办法（暂行）》《横琴粤澳深度合作区支持企业赴澳门发行公司债券专项扶持办法（暂行）》《横琴粤澳深度合作区企业上市挂牌专项扶持办法》《横琴粤澳深度合作区促进中小微企业融资发展扶持办法》《横琴粤澳深度合作区促进金融产业发展扶持办法）》），综合性政策支持企业扎根、专项性政策助力企业成长、业务性政策促进企业展业，横琴金融政策体系初步成型。
做好横琴粤澳深度合作区开发开放，推动前海深港现代服务业合作区全面深化改革开放，有利于进一步推进我国金融开放创新，深化内地与港澳金融合作，为横琴、前海两地建设粤港澳大湾区重大合作平台提供坚实的金融支撑。
《关于金融支持横琴粤澳深度合作区建设的意见》提出，支持横琴粤澳深度合作区面向澳门高度开放，提高外商直接投资汇兑便利化程度等
《关于金融支持横琴粤澳深度合作区建设的意见》发布，紧紧围绕促进澳门经济适度多元发展，推动横琴粤澳深度合作区的金融改革创新和对外合作开放，支持合作区加强与澳门金融市场联通，打造便利澳门居民跨境生活就业的金融环境。
《横琴粤澳深度合作区发展促进条例》从多个方面赋能横琴粤澳深度合作区建设发展，有助于进一步促进澳门经济适度多元发展，打造澳门居民生活就业的新家园。
在横琴粤澳深度合作区管委会的统一部署下，广东金融系统正以推动“横琴金融30条”落地实施为重点，强化对横琴合作区建设的服务支撑。
省地方金融监管局将加快研究出台支持融资租赁行业发展政策，支持横琴合作区利用粤澳合作优势加快发展融资租赁特别是跨境租赁，探索融资租赁资产跨境转让；还将指导横琴合作区完善配套资金补贴等专项政策，支持横琴合作区现代金融产业发展，助力澳门经济适度多元发展。
《关于金融支持横琴粤澳深度合作区建设的意见》明确要求，将横琴粤澳深度合作区打造成为我国金融业对外开放的新高地，中国——葡语国家金融服务平台和粤港澳大湾区金融市场互联互通的示范区
《横琴粤澳深度合作区发展促进条例》政策出台后，将为澳门青年在合作区创新创业提供政策扶持和更多空间,明确具体的税收减免等措施，令大家对横琴的发展充满信心。
《横琴粤澳深度合作区发展促进条例》提到要为澳门青年在合作区创新创业就业

In [115]:
inference_100('目前共有多少名学生在合作区学习？')

1th:  (1, 768)
1th:  (1, 768)

问题：
目前共有多少名学生在合作区学习？

检索结果：
目前，已有230名澳门学生在合作区学习。
合作区现有12所教育机构，其中8所公办学校，4所民办教育机构，搭建合作区多元教育平台。
合作区民生事务局局长黄宇杰致辞时表示，希望学生们通过考察交流，进一步了解合作区的发展战略及未来规划，同时将自身职涯规划与横琴四大产业发展方向相结合，积极参与合作区的建设发展。
为提升澳门青年对合作区发展情况及未来发展规划的认知和了解，合作区民生事务局定期开展名企考察、座谈交流等各类业余活动，让实习生们深入了解到合作区的总体发展规划和民生配套设施等。
合作区民生事务局将加强与澳门相关机构的合作，组织澳门居民到合作区实地考察，组织澳门青年到合作区交流，组织澳门高校学生到合作区实习，让更多澳门居民了解合作区的建设发展，吸引他们到横琴居住就业。

回答：
['230']


# only DPR

In [13]:
inference_100('电子围网系统哪一年建立？',15)

1th:  (1, 768)
1th:  (1, 768)

问题：
电子围网系统哪一年建立？

检索结果：
探索构建电子围网系统。
积极探索支持 " 横琴金融 30 条 " 创新举措落地实施，重点聚焦合作区资金 " 电子围网 "，配合监管机构，尽快推动电子围网相关业务落地，并在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试，助推合作区跨境金融服务向纵深发展。
支持合作区构建特色微电子产业链，建设集成电路先进测试技术和服务平台，布局芯片研发和制造项目，建设全球电子元器件集散中心。
完善开放条件下的金融监管机制，创新基于电子围网系统的跨境金融管理体系，实现跨境资金“管得住、放得开”。
优化改造自由贸易账号系统，通过金融账户隔离，在合作区建立资金电子围网。
“横琴金融30条”是横琴粤澳深度合作区成立以来，由中央部委会同广东省政府发布的首个金融产业创新发展专项政策。此外，还在全国首次推出了一些制度安排，如“构建资金电子围网系统、在合作区探索允许澳门元作为小额支付使用”等，这极大地促进了澳门深度融入国家发展大局。
按照横琴“立足服务澳门、琴澳一体化”的战略定位，设计更加开放的金融制度安排，通过构建电子围网系统，创新跨境金融管理，探索跨境资金自由流动途径。
促进合作区与澳门一体化发展，对标国际标准制定账户规则，通过多功能自由贸易账户构建资金“电子围网”，在合作区探索跨境资金自由流入流出和推进资本项目可兑换。
“横琴金融30条”提出，加强合作区金融市场与澳门、香港离岸金融市场的联动、稳步扩大跨境资产转让范围、稳步开展“跨境理财通”业务试点等具体举措，同时还创新性提出优化改造自由贸易账户（FT账户）系统，通过金融账户隔离，在合作区建立资金电子围网。
《横琴粤澳深度合作区建设总体方案》提出“探索构建电子围网系统，推动合作区金融市场率先高度开放”、“在合作区内探索跨境资本自由流入流出和推进资本项目可兑换”等一系列政策安排，为合作区发展率先高度开放的金融产业提供了政策支撑。
“跨一线”资金自由划转，“跨二线”资金划转视同跨境交易管理。合作区机构“跨二线”同名账户资金划转实行额度管理允许有限“渗透”，资金使用参照负面清单管理。
支持合作区加快发展数字经济，完善数字基础设施，建设人工智能协同创新生态，开展智能医疗、智能驾驶等领域应用，推动大数据、人工智能、物联网等赋能

1th:  (1, 768)
1th:  (1, 768)

问题：
1. 到2025年，电子围网系统和跨境金融管理体系基本建立，合作区与澳门金融服务一体化初步实现，便利澳门居民生活就业的金融环境初步形成，助力澳门经济适度多元的现代金融产业集聚发展，在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试。

检索结果：
到2025年，电子围网系统和跨境金融管理体系基本建立，合作区与澳门金融服务一体化初步实现，便利澳门居民生活就业的金融环境初步形成，助力澳门经济适度多元的现代金融产业集聚发展，在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试。
“横琴金融30条”从打造便利澳门居民生活就业的金融环境、促进合作区与澳门金融市场和金融基础设施互联互通、发展助力澳门经济适度多元发展的现代金融产业、促进跨境贸易和投融资便利化、加强金融监管合作、保障措施等六个方面提出多条具体措施，并提出了2025年、2035年“两步走”的总体目标。
《关于金融支持横琴粤澳深度合作区建设的意见》的出台，为合作区资产管理行业的创新发展提供了更加广阔的空间。一方面，拓宽了“跨境理财通”业务的深度和广度，有助于企业在产品开发、业务拓展和客户服务等多方面提升跨境服务竞争力，更好满足粤澳两地投资者财富管理需求；另一方面，鼓励更多金融机构在合作区设立分支机构及专业子公司，可以优化合作区金融市场业态、激发跨境金融创新活力，为粤澳经济联动和澳门经济多元发展提供更多金融活水。
打造便利澳门居民生活就业的金融环境，促进跨境贸易和投融资便利化。
中央支持合作区金融改革开放的特殊政策，将推动两地业界开展跨境金融创新，例如跨境贷款、跨境双向资金池、跨境资产转让等业务，有序推进合作区与澳门的跨境资本自由流动，支持粤澳跨境投融资便利化，从而吸引一些高增值企业在合作区聚集，作长线的投资发展。
细化落实《横琴意见》措施。其中，以澳门新街坊等民生项目为重点，积极推动惠民生、暖民心举措优先落地实施；注重推进基础性、先导性工作，做好基于现有自由贸易账户（FT账户）体系改造，推动资金“电子围网”落地实施；在与澳门金融市场互联互通、人民币跨境使用、跨境资金流动等领域落实开放措施，提升琴澳金融一体化水平。
持合作区建立与其发展相适应的账户管理体系，推动合作区与澳门资金自由便利流动，建立适应高水平贸易投资自

In [55]:
inference_100('到2025年，电子围网系统和跨境金融管理体系基本建立',15)

1th:  (1, 768)
1th:  (1, 768)

问题：
到2025年，电子围网系统和跨境金融管理体系基本建立

检索结果：
完善开放条件下的金融监管机制，创新基于电子围网系统的跨境金融管理体系，实现跨境资金“管得住、放得开”。
“横琴金融30条”强调金融市场和金融基础设施实现互联互通，建立电子围网，扩大跨境资产转让范围，推动跨境人民币结算等，保障跨境金融工作平稳有序开展。
积极探索支持 " 横琴金融 30 条 " 创新举措落地实施，重点聚焦合作区资金 " 电子围网 "，配合监管机构，尽快推动电子围网相关业务落地，并在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试，助推合作区跨境金融服务向纵深发展。
按照横琴“立足服务澳门、琴澳一体化”的战略定位，设计更加开放的金融制度安排，通过构建电子围网系统，创新跨境金融管理，探索跨境资金自由流动途径。
“横琴金融30条”提出，加强合作区金融市场与澳门、香港离岸金融市场的联动、稳步扩大跨境资产转让范围、稳步开展“跨境理财通”业务试点等具体举措，同时还创新性提出优化改造自由贸易账户（FT账户）系统，通过金融账户隔离，在合作区建立资金电子围网。
促进合作区与澳门一体化发展，对标国际标准制定账户规则，通过多功能自由贸易账户构建资金“电子围网”，在合作区探索跨境资金自由流入流出和推进资本项目可兑换。
到2035年，电子围网系统和跨境金融管理体系更加完善，合作区与澳门金融一体化水平进一步提升，金融环境更加优化，现代金融产业进一步发展。
《横琴粤澳深度合作区建设总体方案》提出“探索构建电子围网系统，推动合作区金融市场率先高度开放”、“在合作区内探索跨境资本自由流入流出和推进资本项目可兑换”等一系列政策安排，为合作区发展率先高度开放的金融产业提供了政策支撑。
落实促进贸易结算便利化、推动跨境人民币结算、提高外商直接投资汇兑便利程度、探索适应市场需求新形态的跨境投资管理体系、优化外债管理、提高跨境证券投融资汇兑便利程度等政策。
到2025年，电子围网系统和跨境金融管理体系基本建立，合作区与澳门金融服务一体化初步实现，便利澳门居民生活就业的金融环境初步形成，助力澳门经济适度多元的现代金融产业集聚发展，在跨境资金流动自由便利、投融资汇兑便利化、金融业对外开放等方面先行先试。
促进贸易结算便利化、推动跨境人民币结算、

In [56]:
inference_100('《横琴粤澳深度合作区发展促进条例》',25)

1th:  (1, 768)
1th:  (1, 768)

问题：
《横琴粤澳深度合作区发展促进条例》

检索结果：
制度保障方面，五个金融政策正式实施（即《横琴粤澳深度合作区外商投资股权投资类企业试点办法（暂行）》《横琴粤澳深度合作区支持企业赴澳门发行公司债券专项扶持办法（暂行）》《横琴粤澳深度合作区企业上市挂牌专项扶持办法》《横琴粤澳深度合作区促进中小微企业融资发展扶持办法》《横琴粤澳深度合作区促进金融产业发展扶持办法）》），综合性政策支持企业扎根、专项性政策助力企业成长、业务性政策促进企业展业，横琴金融政策体系初步成型。
《横琴粤澳深度合作区发展促进条例》明确支持合作区逐步构建民商事规则衔接澳门、接轨国际的制度体系，为港澳律师带来了更大的发展空间和机遇，也为其积极融入国家发展大局提供了更多支撑和保障。
《横琴粤澳深度合作区发展促进条例》的出台将有助于促进生产要素的跨境流通，使澳门青年、创业者来到合作区可以享受与澳门趋同的营商环境，也有助于帮助企业融入国家发展大局。
《横琴粤澳深度合作区发展促进条例》提到要为澳门青年在合作区创新创业就业提供政策扶持，推动在合作区创新创业就业的澳门青年同步享受粤澳两地的扶持政策。
《横琴粤澳深度合作区发展促进条例》是完善澳门居民在合作区内生活、就业、创业制度保障的重要举措，将为澳门居民注入一剂“强心针”，使他们更有信心前往横琴工作生活。
《横琴粤澳深度合作区发展促进条例》政策出台后，将为澳门青年在合作区创新创业提供政策扶持和更多空间,明确具体的税收减免等措施，令大家对横琴的发展充满信心。
《横琴粤澳深度合作区发展促进条例》从多个方面赋能横琴粤澳深度合作区建设发展，有助于进一步促进澳门经济适度多元发展，打造澳门居民生活就业的新家园。
横琴粤澳深度合作区发展促进条例》明确提出“为澳门青年提供更多的创新创业空间；推动在合作区创新创业就业的澳门青年同步享受粤澳两地的扶持政策”，进一步为合作区鼓励澳门青年创新创业工作指明了方向，为今后工作提供了法律保障。
《关于金融支持横琴粤澳深度合作区建设的意见》发布，紧紧围绕促进澳门经济适度多元发展，推动横琴粤澳深度合作区的金融改革创新和对外合作开放，支持合作区加强与澳门金融市场联通，打造便利澳门居民跨境生活就业的金融环境。
《关于金融支持横琴粤澳深度合作区建设的意见》的出台，为合作区资产管理行业的创

# only gen

In [29]:
inference_100('助力澳门经济适度多元的现代金融产业集聚发展',25)

1th:  (1, 768)
1th:  (1, 768)

问题：
助力澳门经济适度多元的现代金融产业集聚发展

检索结果：
澳门、横琴地处粤港澳大湾区，是中国开放程度最高、经济活力最强的地区之一。澳门采取“1+4”产业适度多元发展策略，具备包括自由港、单独关税区等多项营商优势；而合作区正推动科技研发和高端制造、中医药等澳门品牌工业、文旅会展商贸和现代金融等“四新”产业高质量发展。
发展现代金融产业既是澳门经济适度多元发展的重要内容，也是更好地服务文旅会展商贸等产业发展的关键举措，有利于吸纳澳门居民就业，有利于为经济主体提供便捷丰富的金融服务。
随着合作区银行理财公司、金融租赁公司等各类特色金融机构的发展，现代金融业将进一步发挥粤澳深度合作的重要经济纽带作用，为科技创新、高端制造业、中医药、文旅商贸会展等产业发展提供更有力、更便捷、更灵活的金融支持，推动打造促进澳门经济适度多元发展的新平台。
澳门实现“1+4”适度多元发展缺乏资本介入与人才流入，“横琴金融30条”中关于金融配套服务、人才引进就业方面的规定有利于加快合作区与澳门金融服务一体化，加强粤港澳大湾区人员流动。
“工银琴澳通”推出以来，中国工商银行横琴粤澳深度合作区分行不断深化改革创新，为促进澳门经济适度多元发展、建设便利澳门居民生活就业新家园提供全方位、有深度的金融支持。
“横琴金融30条”提到，优先服务民生，在合作区积极营造趋同澳门的宜居宜业金融环境，便利澳门居民生活就业。包括:为符合条件的澳门居民在合作区内购置或出售商品房给予汇兑便利；研究允许合作区内市场主体以澳门元或人民币进行支付；有序允许澳门元在合作区作为小额支付使用等。
合作区建设应当坚持解放思想、改革创新、互利合作、开放包容，发展促进澳门经济适度多元的新产业，建设便利澳门居民生活就业的新家园，构建与澳门一体化高水平开放的新体系，健全粤澳共商共建共管共享的新体制，推动澳门长期繁荣稳定和更好融入国家发展大局。
加强与澳门金融市场联通，推动合作区与澳门金融服务一体化，打造便利澳门居民生活就业的金融环境，发展助力澳门经济适度多元发展的现代金融产业。
金融要加强服务实体经济。当前澳门、合作区的实体产业经济已有一定的基础，在合作区金融发展局的协调推动下，多家金融机构加大力度支持澳门产业多元发展，并为企业“走出去”投融资提供服务，琴澳两地金融机构跨境合作已

# final

In [48]:
inference_100('《横琴粤澳深度合作区发展促进条例》包含什么内容？',25)

1th:  (1, 768)
1th:  (1, 768)

问题：
《横琴粤澳深度合作区发展促进条例》包含什么内容？

检索结果：
制度保障方面，五个金融政策正式实施（即《横琴粤澳深度合作区外商投资股权投资类企业试点办法（暂行）》《横琴粤澳深度合作区支持企业赴澳门发行公司债券专项扶持办法（暂行）》《横琴粤澳深度合作区企业上市挂牌专项扶持办法》《横琴粤澳深度合作区促进中小微企业融资发展扶持办法》《横琴粤澳深度合作区促进金融产业发展扶持办法）》），综合性政策支持企业扎根、专项性政策助力企业成长、业务性政策促进企业展业，横琴金融政策体系初步成型。
《横琴粤澳深度合作区发展促进条例》从多个方面赋能横琴粤澳深度合作区建设发展，有助于进一步促进澳门经济适度多元发展，打造澳门居民生活就业的新家园。
做好横琴粤澳深度合作区开发开放，推动前海深港现代服务业合作区全面深化改革开放，有利于进一步推进我国金融开放创新，深化内地与港澳金融合作，为横琴、前海两地建设粤港澳大湾区重大合作平台提供坚实的金融支撑。
《关于金融支持横琴粤澳深度合作区建设的意见》提出，支持横琴粤澳深度合作区面向澳门高度开放，提高外商直接投资汇兑便利化程度等
《关于金融支持横琴粤澳深度合作区建设的意见》发布，紧紧围绕促进澳门经济适度多元发展，推动横琴粤澳深度合作区的金融改革创新和对外合作开放，支持合作区加强与澳门金融市场联通，打造便利澳门居民跨境生活就业的金融环境。
《横琴粤澳深度合作区发展促进条例》发布，支持合作区发展休闲度假、会议展览、舞台演艺、体育赛事观光、游艇旅游等文旅产业，高水平建设横琴国际休闲旅游岛。
《横琴粤澳深度合作区发展促进条例》政策出台后，将为澳门青年在合作区创新创业提供政策扶持和更多空间,明确具体的税收减免等措施，令大家对横琴的发展充满信心。
《横琴粤澳深度合作区发展促进条例》提到要为澳门青年在合作区创新创业就业提供政策扶持，推动在合作区创新创业就业的澳门青年同步享受粤澳两地的扶持政策。
省地方金融监管局将加快研究出台支持融资租赁行业发展政策，支持横琴合作区利用粤澳合作优势加快发展融资租赁特别是跨境租赁，探索融资租赁资产跨境转让；还将指导横琴合作区完善配套资金补贴等专项政策，支持横琴合作区现代金融产业发展，助力澳门经济适度多元发展。
在横琴粤澳深度合作区管委会的统一部署下，广东金融系统正以推动“横

In [61]:
import pandas as pd

def check_any_element_in_dataframe(elements, dataframe):
    """
    判断列表中的任意一个元素是否存在于DataFrame的['context']列中。

    参数：
    - elements: 要检查的元素组成的列表
    - dataframe: 包含['context']列的DataFrame

    返回：
    如果列表中的任意一个元素存在于DataFrame的['context']列中，返回True；否则返回False。
    """
    # 检查任意一个元素是否在['context']列的值中
    return any(element in dataframe['context'].values for element in elements)

# 示例用法
# 创建一个包含['context']列的DataFrame
data = {'context': ['apple', 'banana', 'orange']}
df = pd.DataFrame(data)

# 要检查的元素列表
elements_to_check = ['a', 'b']

# 调用函数进行检查
result = check_any_element_in_dataframe(elements_to_check, df)

# 打印结果
print(result)


False


In [60]:
df

,context
0,apple
1,banana
2,orange


In [72]:
import pandas as pd

def count_matching_elements_in_dataframe(elements, dataframe):
    """
    计算列表中的任意一个元素是否与DataFrame的某一行的元素相同，并返回匹配的次数。

    参数：
    - elements: 要检查的元素组成的列表
    - dataframe: 包含['context']列的DataFrame

    返回：
    匹配的次数
    """
    count = 0
    for i in range(len(dataframe)):
        row_elements = dataframe.at[i, 'context']
        if any(element in row_elements for element in elements):
            count += 1
    return count

# 示例用法
# 创建一个包含['context']列的DataFrame
data = {'context': ['apple', 'banana', 'orange'],
        'other_column': [10, 20, 30]}
df = pd.DataFrame(data)

# 要检查的元素列表
elements_to_check = ['banana', 'apple','orange','asd']

# 调用函数进行计数
count = count_matching_elements_in_dataframe(elements_to_check, df)

# 打印计数结果
print(count)


3


In [70]:
df

,context,other_column
0,apple,10
1,banana,20
2,orange,30


In [1]:
from datasets import load_from_disk
# print(f'DEBUGGER: 读取kb中，path： {passages_path}')
dataset = load_from_disk('/root/autodl-tmp/rag-end2end-retriever/DIR/baiduzhidao/squad2w')  # to reload the dataset

/root/miniconda3/envs/rag/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

Dataset({
    features: ['text', 'title', 'embeddings'],
    num_rows: 18340
})

In [5]:
a=0
for i in range(18340):
    a += len(dataset[i]['text'])

In [6]:
a

4061807